In [1]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", "")
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    df = pd.DataFrame(all_data)
    df.drop(columns=["Image"], inplace=True)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

def prompt_select_files(event=None):
    read_dicom_files()

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

def animate_label_color(label, colors, index=0):
    label.configure(text_color=colors[index])
    next_index = (index + 1) % len(colors)
    label.after(500, animate_label_color, label, colors, next_index)

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    if not all_data:
        label = ctk.CTkLabel(
            content_frame,
            text="\n Click Here To Select DICOM Files",
            font=ctk.CTkFont(size=28, weight="bold"),
            justify="center",
            anchor="center",
            cursor="hand2"
        )
        label.pack(expand=True, fill="both", padx=50, pady=50)
        label.bind("<Button-1>", lambda e: read_dicom_files())

        # ألوان مريحة تتغير تدريجياً
        colors = ["#3a86ff", "#4dabf5", "#71b8ff", "#4dabf5"]
        animate_label_color(label, colors)

        return


    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])
    dose_summary = defaultdict(lambda: defaultdict(float))
    date_ranges = defaultdict(lambda: defaultdict(list))

    columns = 4
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            ctk.CTkLabel(frame, text="No Image").pack()

        info = (
            f"👤 Name: {data['Name']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"☢ Dose Result (mSv): {data['mSv']:.2f}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

        start_date = data["Date"]
        end_date = start_date + timedelta(days=365)
        patient_name = data["Name"]
        for d in all_data:
            if d["Name"] == patient_name and start_date <= d["Date"] <= end_date:
                year = start_date.year
                dose_summary[patient_name][year] += d["mSv"]
                date_ranges[patient_name][year].append(d["Date"])

    summary_box.delete("1.0", "end")
    summary_box.insert("end", "\n📊 Sum of Dose per Year:\n")
    for name in dose_summary:
        for year, total_dose in dose_summary[name].items():
            dates = sorted(date_ranges[name][year])
            start = dates[0].strftime("%Y-%m-%d") if dates else "N/A"
            end = dates[-1].strftime("%Y-%m-%d") if dates else "N/A"
            summary_box.insert("end", f"{name} - {year}: {total_dose:.2f} mSv (From {start} to {end})\n")

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            return

    messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
    hl7_textbox.delete("1.0", "end")

def clear_images():
    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()
    summary_box.delete("1.0", "end")
    hl7_textbox.delete("1.0", "end")
    display_images()  # اعادة عرض رسالة رفع الملفات

root = ctk.CTk()
root.title("DICOM Viewer + HL7 + Summary")
root.geometry("1250x850")
root.resizable(True, True)

try:
    bg_image = Image.open("g.jpg").resize((1250, 850))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")
    bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not loaded: {e}")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.05)

clear_button = ctk.CTkButton(root, text="🗑 Clear All", command=clear_images)
clear_button.place(relx=0.15, rely=0.05)

ctk.CTkLabel(root, text="🔍 Search HL7 by Name:").place(relx=0.35, rely=0.05)
search_entry = ctk.CTkEntry(root, width=180)
search_entry.place(relx=0.52, rely=0.05)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(relx=0.68, rely=0.05)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.8, rely=0.05)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(relx=0.85, rely=0.05)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.12, relwidth=0.96, relheight=0.55)

hl7_textbox = ctk.CTkTextbox(root)
hl7_textbox.place(relx=0.02, rely=0.7, relwidth=0.96, relheight=0.1)

summary_box = ctk.CTkTextbox(root)
summary_box.place(relx=0.02, rely=0.82, relwidth=0.96, relheight=0.1)

display_images()  # عند بداية التشغيل تظهر رسالة رفع الملفات

root.mainloop()


C:\Users\maryam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


In [ ]:
# DICOM HL7 Viewer Advanced

import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

CSV_FILE = "patients_data.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# ========== HL7 Message Generator ==========
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", ""),
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "PatientID", "")
    modality = getattr(ds, "Modality", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{study_id}||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

# ========== File Handling ==========
def read_single_dicom():
    file = filedialog.askopenfilename(filetypes=[("DICOM files", "*.dcm")])
    if not file:
        return

    try:
        ds = pydicom.dcmread(file)
        name = str(getattr(ds, "PatientName", "Unknown"))
        date_str = getattr(ds, "StudyDate", "00000000")
        date_obj = datetime.strptime(date_str, "%Y%m%d") if date_str != "" else datetime.now()
        ctdi = float(getattr(ds, "CTDIvol", 0))
        modality = getattr(ds, "Modality", "Unknown")
        msv = ctdi * 0.014
        patient_id = getattr(ds, "PatientID", "")

        row = {
            "ID": patient_id,
            "Name": name,
            "Date": date_obj,
            "Modality": modality,
            "CTDIvol": ctdi,
            "mSv": msv
        }

        df = pd.DataFrame([row])
        if os.path.exists(CSV_FILE):
            df.to_csv(CSV_FILE, mode='a', index=False, header=False)
        else:
            df.to_csv(CSV_FILE, index=False)

        hl7_msg = convert_to_hl7(ds, msv)
        hl7_filename = f"{HL7_DIR}/{patient_id}_{date_obj.strftime('%Y%m%d')}.hl7"
        with open(hl7_filename, "w") as f:
            f.write(hl7_msg)

        messagebox.showinfo("Success", "DICOM file imported and data saved.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to process file.\n{e}")

# ========== Viewer ==========
def read_multiple_dicom():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return
    all_data.clear()
    for f in files:
        try:
            ds = pydicom.dcmread(f)
            name = str(getattr(ds, "PatientName", "Unknown"))
            date = getattr(ds, "StudyDate", "00000000")
            date_obj = datetime.strptime(date, "%Y%m%d")
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            modality = getattr(ds, "Modality", "Unknown")
            msv = ctdi * 0.014
            img = Image.fromarray(ds.pixel_array)
            img.thumbnail((200, 200))
            img_tk = ImageTk.PhotoImage(img)

            all_data.append({
                "ID": getattr(ds, "PatientID", ""),
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "mSv": msv,
                "Modality": modality,
                "Image": img_tk
            })
        except Exception as e:
            print(f"Error: {e}")

    display_cases()

def display_cases():
    for widget in cases_frame.winfo_children():
        widget.destroy()

    sorted_data = sorted(all_data, key=lambda x: x[sort_var.get()])

    for i, d in enumerate(sorted_data[:4]):
        frame = ctk.CTkFrame(cases_frame)
        frame.grid(row=i//2, column=i%2, padx=10, pady=10)
        ctk.CTkLabel(frame, text=f"ID: {d['ID']}\nName: {d['Name']}\nDate: {d['Date'].date()}\nModality: {d['Modality']}\nCTDIvol: {d['CTDIvol']} mGy\nmSv: {d['mSv']:.2f}").pack()
        ctk.CTkLabel(frame, image=d["Image"], text="").pack()

# ========== HL7 Search ==========
def search_hl7():
    pwd = simpledialog.askstring("Password", "Enter password:", show='*')
    if pwd != "admin123":
        messagebox.showerror("Access Denied", "Incorrect password")
        return

    pid = simpledialog.askstring("Search HL7", "Enter Patient ID:")
    for fname in os.listdir(HL7_DIR):
        if pid in fname:
            with open(os.path.join(HL7_DIR, fname)) as f:
                hl7_view.delete("1.0", "end")
                hl7_view.insert("end", f.read())
                return
    messagebox.showinfo("Not Found", "No HL7 message found for this ID.")

# ========== GUI Layout ==========
root = ctk.CTk()
root.geometry("1200x850")
root.title("Advanced DICOM Viewer + HL7")

ctk.CTkButton(root, text="📥 Import Single DICOM", command=read_single_dicom).place(relx=0.03, rely=0.02)
ctk.CTkButton(root, text="📂 Select Cases", command=read_multiple_dicom).place(relx=0.25, rely=0.02)
ctk.CTkButton(root, text="🔐 HL7 Messages", command=search_hl7).place(relx=0.45, rely=0.02)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort By:").place(relx=0.65, rely=0.02)
ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_cases()).place(relx=0.72, rely=0.02)

cases_frame = ctk.CTkScrollableFrame(root)
cases_frame.place(relx=0.03, rely=0.08, relwidth=0.94, relheight=0.6)

hl7_view = ctk.CTkTextbox(root)
hl7_view.place(relx=0.03, rely=0.7, relwidth=0.94, relheight=0.25)

root.mainloop()


In [ ]:
# هذا الكود هو تكملة وتطوير للمشروع السابق مع دعم اختيار حالتين أو أربع حالات لعرضهم
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

all_data = []
selected_cases = []

# ---------- الوظائف ----------
def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            study_type = str(getattr(ds, "Modality", "Unknown"))
            date_str = getattr(ds, "StudyDate", "00000000")
            study_id = str(getattr(ds, "PatientID", "Unknown"))
            gender = getattr(ds, "PatientSex", "")
            dob = getattr(ds, "PatientBirthDate", "")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((150, 150))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            data = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "mSv": msv,
                "StudyType": study_type,
                "Image": photo,
                "Sex": gender,
                "DOB": dob,
                "ID": study_id,
                "Path": path
            }

            all_data.append(data)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_cases()

def display_cases():
    for widget in content_frame.winfo_children():
        widget.destroy()

    for i, data in enumerate(all_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=i//3, column=i%3, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 {data['Name']}\n"
            f"🆔 ID: {data['ID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧪 Type: {data['StudyType']}\n"
            f"☢ CTDIvol: {data['CTDIvol']} mGy\n"
            f"☢ mSv: {data['mSv']:.2f} mSv"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack()

        chk = ctk.CTkCheckBox(frame, text="اختيار الحالة",
                              command=lambda d=data: toggle_selection(d))
        chk.pack(pady=5)

def toggle_selection(data):
    if data in selected_cases:
        selected_cases.remove(data)
    elif len(selected_cases) < 4:
        selected_cases.append(data)
    else:
        messagebox.showwarning("Max Reached", "يمكنك فقط اختيار حالتين إلى أربع حالات.")

def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "يجب اختيار حالتين أو أربع حالات فقط.")
        return

    win = ctk.CTkToplevel(root)
    win.title("📋 الحالات المختارة")
    win.geometry("900x600")

    for i, data in enumerate(selected_cases):
        frame = ctk.CTkFrame(win)
        frame.grid(row=i//2, column=i%2, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        details = (
            f"👤 الاسم: {data['Name']}\n"
            f"🆔 ID: {data['ID']}\n"
            f"📅 التاريخ: {data['Date'].date()}\n"
            f"🎯 النوع: {data['StudyType']}\n"
            f"☢ CTDIvol: {data['CTDIvol']} mGy\n"
            f"☢ الجرعة (mSv): {data['mSv']:.2f} mSv\n"
            f"👤 النوع: {data['Sex']}\n"
            f"🎂 تاريخ الميلاد: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=details, justify="left").pack(pady=5)

# ---------- واجهة البرنامج ----------
root = ctk.CTk()
root.title("📁 DICOM Viewer & Selector")
root.geometry("1280x800")

select_button = ctk.CTkButton(root, text="📂 اختر ملفات DICOM", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.86)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


# قراءة ملفات DICOM وتخزين البيانات فقط

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


# عرض البيانات فقط بدون صور

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])
    
    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=10)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=5)

        delete_btn = ctk.CTkButton(frame, text="🗑 Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.pack(side="right", padx=5)

        hl7_btn = ctk.CTkButton(frame, text="📨 HL7 Message", width=110,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.pack(side="right", padx=5)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    if data in selected_cases:
        selected_cases.remove(data)
    display_text_data()


def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=5)

        delete_btn = ctk.CTkButton(frame, text="🗑 Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.pack(side="right", padx=5)

        hl7_btn = ctk.CTkButton(frame, text="📨 HL7 Message", width=110,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.pack(side="right", padx=5)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    if data in selected_cases:
        selected_cases.remove(data)
    display_text_data()


def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog, IntVar
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_vars = []  # to hold IntVars for checkboxes

# Convert DICOM to HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_vars.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table_data()

def display_table_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # headers
    headers = ["Select", "Name", "ID", "Date", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB", "HL7", "Delete"]

    # Sort data
    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # Create header labels
    for col, header in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="nsew")

    # Clear selected_vars and recreate
    selected_vars.clear()

    for row, data in enumerate(sorted_data, start=1):
        var = IntVar()
        selected_vars.append((var, data))
        checkbox = ctk.CTkCheckBox(content_frame, variable=var)
        checkbox.grid(row=row, column=0, padx=5, pady=5, sticky="nsew")

        ctk.CTkLabel(content_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["StudyID"]).grid(row=row, column=2, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=3, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=4, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']}").grid(row=row, column=5, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['DLP']}").grid(row=row, column=6, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row, column=7, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row, column=8, padx=5, pady=5, sticky="nsew")

        hl7_btn = ctk.CTkButton(content_frame, text="View HL7", width=80,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.grid(row=row, column=9, padx=5, pady=5)

        delete_btn = ctk.CTkButton(content_frame, text="Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.grid(row=row, column=10, padx=5, pady=5)

def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    # Also remove from selected_vars any var related to this data
    for var, d in selected_vars[:]:
        if d == data:
            selected_vars.remove((var, d))
    display_table_data()

def show_selected_cases():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.attributes('-topmost', True)  # Keep window on top

    for idx, data in enumerate(selected):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=14)).pack(pady=5, padx=10)

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1300x900")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_table_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog, ttk
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_dict = {}  # مفتاح: (Name, Date) -> بيانات الحالة بدون تكرار

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            key = (name, date_obj.date())  # فقط التاريخ بدون وقت

            if key not in temp_dict:
                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path
                }

                temp_dict[key] = data_dict

                # حفظ HL7
                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_dict.values())
    display_text_data()

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # زر HL7 فوق الجدول
    hl7_all_btn = ctk.CTkButton(content_frame, text="📨 Show HL7 Messages for Selected", command=show_hl7_for_selected)
    hl7_all_btn.pack(pady=5)

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # إنشاء جدول
    columns = ("Name", "Date", "StudyID", "Dose_mSv", "CTDIvol", "DLP", "Sex", "DOB", "Select")
    tree = ttk.Treeview(content_frame, columns=columns, show='headings', selectmode='none')
    tree.pack(fill="both", expand=True)

    # تهيئة رؤوس الأعمدة
    tree.heading("Name", text="Name")
    tree.heading("Date", text="Date")
    tree.heading("StudyID", text="Study ID")
    tree.heading("Dose_mSv", text="Dose (mSv)")
    tree.heading("CTDIvol", text="CTDIvol (mGy)")
    tree.heading("DLP", text="DLP (mGy·cm)")
    tree.heading("Sex", text="Sex")
    tree.heading("DOB", text="DOB")
    tree.heading("Select", text="Select")

    # ضبط عرض الأعمدة
    tree.column("Name", width=150)
    tree.column("Date", width=100)
    tree.column("StudyID", width=80)
    tree.column("Dose_mSv", width=90, anchor="center")
    tree.column("CTDIvol", width=90, anchor="center")
    tree.column("DLP", width=90, anchor="center")
    tree.column("Sex", width=50, anchor="center")
    tree.column("DOB", width=100)
    tree.column("Select", width=70, anchor="center")

    # قائمة للتحكم بالcheckbox
    check_vars.clear()
    for i, data in enumerate(sorted_data):
        var = ctk.BooleanVar()
        check_vars.append(var)

        # تاريخ فقط بدون وقت
        date_only = data['Date'].date().strftime("%Y-%m-%d")
        tree.insert("", "end", iid=str(i), values=(
            data['Name'],
            date_only,
            data['StudyID'],
            f"{data['mSv']:.2f}",
            f"{data['CTDIvol']}",
            f"{data['DLP']}",
            data['Sex'],
            data['DOB'],
            ""
        ))
        # وضع الـ checkbox داخل العمود 'Select' ممكن يكون معقد مع ttk مباشرة،
        # لذا سنضيف checkboxes بشكل منفصل تحت الجدول.

    # وضع Checkboxes منفصلة تحت الجدول (لأن ttk لا يدعم checkbox داخل الخانات بسهولة)
    checkbox_frame = ctk.CTkFrame(content_frame)
    checkbox_frame.pack(pady=5, fill="x")
    for i, data in enumerate(sorted_data):
        cb = ctk.CTkCheckBox(checkbox_frame, text=f"Select Case {i+1}", variable=check_vars[i])
        cb.grid(row=0, column=i, padx=10)

def show_hl7_for_selected():
    # تجميع الحالات المختارة حسب checkbox
    selected_cases.clear()
    for idx, var in enumerate(check_vars):
        if var.get():
            selected_cases.append(all_data[idx])

    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select cases first.")
        return
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    # فتح النافذة أمام النافذة الرئيسية
    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.transient(root)
    top.grab_set()
    top.focus_force()

    # عرض الصور والمعلومات بشكل منسق
    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10, border_width=1, border_color="gray")
        frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"Name: {data['Name']}\n"
            f"ID: {data['StudyID']}\n"
            f"Date: {data['Date'].date()}\n"
            f"Type: CT\n"
            f"Dose: {data['mSv']:.2f} mSv\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Sex: {data['Sex']}\n"
            f"DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

check_vars = []

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

view_button = ctk.CTkButton(root, text="👁‍🗨 View Selected Cases", command=show_hl7_for_selected)
view_button.place(relx=0.22, rely=0.03)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = {}

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    seen = set()  # لتجنب تكرار نفس الحالة بنفس StudyID و Date

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014
            modality = getattr(ds, "Modality", "Unknown")
            study_id = getattr(ds, "StudyID", "")
            accession = getattr(ds, "AccessionNumber", "")

            # المفتاح لتمييز الحالات المكررة: (Name, date_str, study_id, accession)
            unique_key = (name, date_str, study_id, accession)
            if unique_key in seen:
                continue
            seen.add(unique_key)

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": study_id,
                "Accession": accession,
                "Modality": modality,
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            # حفظ HL7
            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{study_id}_{accession}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table()


def display_table():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # عناوين الأعمدة
    headers = ["Select", "Name", "Date", "Modality", "Study ID", "Accession", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB"]

    for col, header in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=header, width=110, anchor="center", font=ctk.CTkFont(weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="ew")

    # ترتيب البيانات
    sort_key = sort_var.get()
    if sort_key == "Date":
        sorted_data = sorted(all_data, key=lambda x: x["Date"])
    else:
        sorted_data = sorted(all_data, key=lambda x: x["Name"].lower())

    for row_idx, data in enumerate(sorted_data, start=1):
        # Checkbox للـ select
        var = ctk.BooleanVar(value=(data in selected_cases))
        def on_check(d=data, v=var):
            if v.get():
                if len(selected_cases) >= 4:
                    messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
                    v.set(False)
                    return
                selected_cases[d] = True
            else:
                if d in selected_cases:
                    del selected_cases[d]

        chk = ctk.CTkCheckBox(content_frame, variable=var, command=on_check)
        chk.grid(row=row_idx, column=0, padx=5, pady=5, sticky="ew")

        ctk.CTkLabel(content_frame, text=data["Name"], anchor="w").grid(row=row_idx, column=1, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d"), anchor="w").grid(row=row_idx, column=2, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["Modality"], anchor="w").grid(row=row_idx, column=3, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["StudyID"], anchor="w").grid(row=row_idx, column=4, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["Accession"], anchor="w").grid(row=row_idx, column=5, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}", anchor="e").grid(row=row_idx, column=6, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']}", anchor="e").grid(row=row_idx, column=7, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=f"{data['DLP']}", anchor="e").grid(row=row_idx, column=8, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["Sex"], anchor="w").grid(row=row_idx, column=9, padx=5, pady=5, sticky="ew")
        ctk.CTkLabel(content_frame, text=data["DOB"], anchor="w").grid(row=row_idx, column=10, padx=5, pady=5, sticky="ew")


def show_hl7_messages():
    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 messages.")
        return

    combined_hl7 = ""
    for data in selected_cases.keys():
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}_{data['Accession']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                combined_hl7 += f"\n--- HL7 Message for {data['Name']} ({data['Date'].strftime('%Y-%m-%d')}) ---\n"
                combined_hl7 += f.read() + "\n"
        else:
            combined_hl7 += f"\n--- HL7 Message for {data['Name']} ({data['Date'].strftime('%Y-%m-%d')}) NOT FOUND ---\n"

    hl7_win = ctk.CTkToplevel()
    hl7_win.title("HL7 Messages")
    hl7_win.geometry("700x500")
    txt = ctk.CTkTextbox(hl7_win)
    txt.pack(fill="both", expand=True)
    txt.insert("0.0", combined_hl7)
    txt.configure(state="disabled")


def show_selected_cases():
    count = len(selected_cases)
    if count not in [2,4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.attributes("-topmost", True)  # تظهر أمام النافذة الرئيسية

    for idx, data in enumerate(selected_cases.keys()):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, width=530, height=320, corner_radius=10)
        frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")
        frame.grid_propagate(False)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"Name: {data['Name']}\n"
            f"ID: {data['StudyID']}\n"
            f"Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"Type: {data['Modality']}\n"
            f"Dose: {data['mSv']:.2f} mSv\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Sex: {data['Sex']}\n"
            f"DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

hl7_button = ctk.CTkButton(root, text="📨 Show HL7 Messages", command=show_hl7_messages)
hl7_button.place(relx=0.22, rely=0.03)

view_button = ctk.CTkButton(root, text="👁 Show Selected Cases", command=show_selected_cases)
view_button.place(relx=0.42, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_table())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []  # مهم جداً لتخزين الصور حتى لا يتم جمعها من الذاكرة


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    global image_refs
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    image_refs.clear()

    # تجميع البيانات مع دمج الحالات المتكررة بنفس الاسم والتاريخ والنوع
    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                try:
                    arr = ds.pixel_array
                    if len(arr.shape) == 3 and arr.shape[0] == 3:
                        arr = arr.transpose(1, 2, 0)
                    pil_img = Image.fromarray(arr)
                    pil_img.thumbnail((150, 150))
                    img = ImageTk.PhotoImage(pil_img)
                    image_refs.append(img)
                except:
                    img = None

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            # نأخذ أول حالة فقط إذا تكررت بنفس المفاتيح
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Image": img,
                    "Path": path,
                    "Ds": ds
                }

                # حفظ رسالة HL7
                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # رأس الجدول
    headers = ["Select", "Name", "Date", "Modality", "CTDIvol (mGy)", "DLP (mGy·cm)", "Dose (mSv)", "Sex", "DOB"]
    header_frame = ctk.CTkFrame(content_frame)
    header_frame.grid(row=0, column=0, sticky="ew", padx=10, pady=5)
    for idx, h in enumerate(headers):
        lbl = ctk.CTkLabel(header_frame, text=h, font=ctk.CTkFont(size=12, weight="bold"))
        lbl.grid(row=0, column=idx, padx=5)

    # عرض البيانات
    for index, data in enumerate(sorted_data, start=1):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, sticky="ew", padx=10, pady=2)

        var = ctk.BooleanVar(value=(data in selected_cases))

        checkbox = ctk.CTkCheckBox(frame, variable=var,
                                   command=lambda d=data, v=var: on_check(d, v))
        checkbox.grid(row=0, column=0, padx=5)

        ctk.CTkLabel(frame, text=data['Name'], width=150, anchor="w").grid(row=0, column=1, padx=5)
        ctk.CTkLabel(frame, text=data['Date'].strftime("%Y-%m-%d"), width=100).grid(row=0, column=2, padx=5)
        ctk.CTkLabel(frame, text=data['Modality'], width=100).grid(row=0, column=3, padx=5)
        ctk.CTkLabel(frame, text=f"{data['CTDIvol']}", width=100).grid(row=0, column=4, padx=5)
        ctk.CTkLabel(frame, text=f"{data['DLP']}", width=100).grid(row=0, column=5, padx=5)
        ctk.CTkLabel(frame, text=f"{data['mSv']:.2f}", width=100).grid(row=0, column=6, padx=5)
        ctk.CTkLabel(frame, text=data['Sex'], width=50).grid(row=0, column=7, padx=5)
        ctk.CTkLabel(frame, text=data['DOB'], width=100).grid(row=0, column=8, padx=5)


def on_check(data, var):
    if var.get():
        if data in selected_cases:
            return
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
            var.set(False)
            return
        selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def delete_selected_cases():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    for case in selected_cases[:]:
        if case in all_data:
            all_data.remove(case)
        selected_cases.remove(case)
    display_text_data()


def show_hl7_messages():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in selected_cases:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    # نافذة عرض الرسائل
    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()  # لجعل النافذة تظهر أمام كل النوافذ الأخرى

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Modality: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

delete_button = ctk.CTkButton(root, text="🗑 Delete Selected", command=delete_selected_cases)
delete_button.place(relx=0.38, rely=0.03)

hl7_button = ctk.CTkButton(root, text="📨 Show HL7 Messages", command=show_hl7_messages)
hl7_button.place(relx=0.55, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []  # مهم جداً لتخزين الصور حتى لا يتم جمعها من الذاكرة


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    global image_refs
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    image_refs.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                try:
                    arr = ds.pixel_array
                    if len(arr.shape) == 3 and arr.shape[0] == 3:
                        arr = arr.transpose(1, 2, 0)
                    pil_img = Image.fromarray(arr)
                    pil_img.thumbnail((120, 120))
                    img = ImageTk.PhotoImage(pil_img)
                    image_refs.append(img)
                except:
                    img = None

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Image": img,
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # إنشاء إطار لكل عمود عموديًا
    columns = {
        "Select": [],
        "Image": [],
        "Name": [],
        "Date": [],
        "Modality": [],
        "CTDIvol (mGy)": [],
        "DLP (mGy·cm)": [],
        "Dose (mSv)": [],
        "Sex": [],
        "DOB": []
    }

    # رؤوس الأعمدة
    header_font = ctk.CTkFont(size=14, weight="bold")
    for col_idx, col_name in enumerate(columns.keys()):
        lbl = ctk.CTkLabel(content_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=8, pady=5)

    # ملء البيانات لكل عمود في عموده الخاص
    for row_idx, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=(data in selected_cases))

        # خانة الاختيار
        chk = ctk.CTkCheckBox(content_frame, variable=var,
                              command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=row_idx, column=0, padx=8, pady=6)

        # صورة صغيرة إن وجدت
        if data["Image"]:
            img_lbl = ctk.CTkLabel(content_frame, image=data["Image"], text="")
            img_lbl.image = data["Image"]
            img_lbl.grid(row=row_idx, column=1, padx=8, pady=6)
        else:
            img_lbl = ctk.CTkLabel(content_frame, text="No Image", fg_color="#ddd", width=120, height=120)
            img_lbl.grid(row=row_idx, column=1, padx=8, pady=6)

        ctk.CTkLabel(content_frame, text=data["Name"], anchor="w").grid(row=row_idx, column=2, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row_idx, column=3, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=data["Modality"]).grid(row=row_idx, column=4, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']:.2f}").grid(row=row_idx, column=5, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=f"{data['DLP']:.2f}").grid(row=row_idx, column=6, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row_idx, column=7, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row_idx, column=8, padx=8, pady=6)
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row_idx, column=9, padx=8, pady=6)


def on_check(data, var):
    if var.get():
        if data in selected_cases:
            return
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
            var.set(False)
            return
        selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def delete_selected_cases():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    for case in selected_cases[:]:
        if case in all_data:
            all_data.remove(case)
        selected_cases.remove(case)
    display_text_data()


def show_hl7_messages():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in selected_cases:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()  # النافذة تظهر قدام النوافذ الأخرى

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=15, pady=15)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Modality: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']:.2f} mGy\n"
            f"📏 DLP: {data['DLP']:.2f} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=13)).pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Elegant Display")
root.geometry("1300x900")

# إضافة صورة الخلفية
bg_img = Image.open("background.jpg").resize((1300, 900))
bg_img_tk = ImageTk.PhotoImage(bg_img)
background_label = ctk.CTkLabel(root, image=bg_img_tk)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files,
                              font=ctk.CTkFont(size=16, weight="bold"))
select_button.pack(pady=12)

sort_var = ctk.StringVar(value="Name")
sort_frame = ctk.CTkFrame(root)
sort_frame.pack(pady=6)
ctk.CTkLabel(sort_frame, text="Sort by:", font=ctk.CTkFont(size=14)).pack(side="left", padx=8)
ctk.CTkOptionMenu(sort_frame, variable=sort_var, values=["Name", "Date"], command=lambda e: display_text_data()).pack(side="left")

hl7_btn = ctk.CTkButton(root, text="Show HL7 Messages", command=show_hl7_messages,
                        font=ctk.CTkFont(size=14, weight="bold"))
hl7_btn.pack(pady=6)

content_frame = ctk.CTkFrame(root, fg_color="#f8f9fa", corner_radius=15)
content_frame.pack(expand=True, fill="both", padx=12, pady=12)

btn_frame = ctk.CTkFrame(root)
btn_frame.pack(pady=10)
ctk.CTkButton(btn_frame, text="Delete Selected", command=delete_selected_cases).pack(side="left", padx=12)
ctk.CTkButton(btn_frame, text="View Selected Cases", command=show_selected_cases).pack(side="left", padx=12)

root.mainloop()


In [ ]:
import os
import pydicom
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import csv
import base64

# --- إعداد الواجهة ---
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

# --- المتغيرات العامة ---
cases_data = []

# --- تحميل وتحليل ملفات DICOM ---
def load_dicom_files():
    filenames = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    for file_path in filenames:
        try:
            ds = pydicom.dcmread(file_path)
            img = ds.pixel_array
            img = Image.fromarray(img).resize((180, 180))
            photo = ImageTk.PhotoImage(image=img)

            patient_name = getattr(ds, "PatientName", "N/A")
            study_date = getattr(ds, "StudyDate", "N/A")
            patient_id = getattr(ds, "PatientID", "N/A")
            ctdi = getattr(ds, "CTDIvol", 0)

            cases_data.append({
                "image": photo,
                "photo_obj": img,
                "name": str(patient_name),
                "date": study_date,
                "ctdi": float(ctdi),
                "id": patient_id,
                "path": file_path,
                "selected": ctk.BooleanVar()
            })
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file:\n{file_path}\n{str(e)}")

    save_to_csv()
    display_text_data()

# --- حفظ البيانات في CSV ---
def save_to_csv():
    with open("rad.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Patient Name", "Patient ID", "Study Date", "CTDIvol"])
        for c in cases_data:
            writer.writerow([c["name"], c["id"], c["date"], c["ctdi"]])

# --- عرض البيانات ---
def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_key = sort_var.get().lower()
    sorted_cases = sorted(cases_data, key=lambda x: x[sort_key])

    for case in sorted_cases:
        frame = ctk.CTkFrame(content_frame)
        frame.pack(pady=5, padx=5, fill="x")

        img_label = ctk.CTkLabel(frame, image=case["image"], text="")
        img_label.pack(side="left", padx=5)

        info = f"Name: {case['name']}\nDate: {case['date']}\nCTDIvol: {case['ctdi']}\nID: {case['id']}"
        ctk.CTkLabel(frame, text=info, anchor="w", justify="left").pack(side="left", padx=10)

        checkbox = ctk.CTkCheckBox(frame, text="Select", variable=case["selected"])
        checkbox.pack(side="right", padx=10)

# --- عرض الحالات المحددة (2 أو 4) ---
def show_selected_cases():
    selected = [c for c in cases_data if c["selected"].get()]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    window = ctk.CTkToplevel()
    window.title("Selected Cases")

    rows = 2 if len(selected) == 4 else 1
    cols = len(selected) // rows

    for i, case in enumerate(selected):
        frame = ctk.CTkFrame(window)
        frame.grid(row=i // cols, column=i % cols, padx=10, pady=10)

        img = ImageTk.PhotoImage(case["photo_obj"].resize((256, 256)))
        ctk.CTkLabel(frame, image=img, text="").pack()
        frame.image = img

        info = f"{case['name']}\n{case['date']}\nCTDIvol: {case['ctdi']}"
        ctk.CTkLabel(frame, text=info, justify="left").pack()

# --- توليد رسالة HL7 مشفرة ---
def encode_hl7(case):
    hl7 = f"""MSH|^~\&|DICOM|Radiology|HL7Viewer|Hospital|{case['date']}||ORU^R01|1234|P|2.3.1
PID|||{case['id']}||{case['name']}|||
OBR|||1^CT Scan|||{case['date']}
OBX|||CTDIvol||{case['ctdi']}|mGy"""
    return base64.b64encode(hl7.encode()).decode()

def show_hl7_messages():
    password = ctk.CTkInputDialog(text="Enter admin password:", title="HL7 Viewer").get_input()
    if password != "admin123":
        messagebox.showerror("Access Denied", "Wrong password.")
        return

    window = ctk.CTkToplevel()
    window.title("HL7 Messages")

    for case in cases_data:
        hl7_encoded = encode_hl7(case)
        frame = ctk.CTkFrame(window)
        frame.pack(fill="x", padx=5, pady=5)

        info = f"{case['name']} - {case['date']}"
        ctk.CTkLabel(frame, text=info).pack(anchor="w")

        text = ctk.CTkTextbox(frame, height=60)
        text.insert("1.0", hl7_encoded)
        text.pack(fill="x")

# --- حذف الحالات المحددة ---
def delete_selected_cases():
    global cases_data
    cases_data = [c for c in cases_data if not c["selected"].get()]
    save_to_csv()
    display_text_data()

# --- زر تحميل ---
def create_colored_label():
    button = ctk.CTkButton(root, text="Load DICOM Files", command=load_dicom_files)
    button.pack(pady=5)

# --- الواجهة الرئيسية ---
root = ctk.CTk()
root.title("DICOM Viewer & Analyzer")
root.geometry("1280x800")

top_frame = ctk.CTkFrame(root)
top_frame.pack(fill="x", padx=15, pady=8)

sort_var = ctk.StringVar(value="name")
ctk.CTkLabel(top_frame, text="Sort by:").pack(side="left", padx=5)
ctk.CTkOptionMenu(top_frame, variable=sort_var, values=["name", "date"], command=lambda _: display_text_data()).pack(side="left", padx=5)

ctk.CTkButton(top_frame, text="Delete Selected", command=delete_selected_cases).pack(side="left", padx=15)
ctk.CTkButton(top_frame, text="Show HL7", command=show_hl7_messages).pack(side="left", padx=5)
ctk.CTkButton(top_frame, text="Show Selected Cases", command=show_selected_cases).pack(side="left", padx=5)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.pack(expand=True, fill="both", padx=15, pady=10)

create_colored_label()
root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    global image_refs
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    image_refs.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                try:
                    arr = ds.pixel_array
                    if len(arr.shape) == 3 and arr.shape[0] == 3:
                        arr = arr.transpose(1, 2, 0)
                    pil_img = Image.fromarray(arr)
                    pil_img.thumbnail((120, 120))
                    img = ImageTk.PhotoImage(pil_img)
                    image_refs.append(img)
                except:
                    img = None

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Image": img,
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    columns = {
        "Select": [],
        "Image": [],
        "Name": [],
        "Date": [],
        "Modality": [],
        "CTDIvol (mGy)": [],
        "DLP (mGy·cm)": [],
        "Dose (mSv)": [],
        "Sex": [],
        "DOB": []
    }

    header_font = ctk.CTkFont(size=14, weight="bold")
    for col_idx, col_name in enumerate(columns.keys()):
        lbl = ctk.CTkLabel(scrollable_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=8, pady=6, sticky="nsew")

    for row_idx, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=(data in selected_cases))
        chk = ctk.CTkCheckBox(scrollable_frame, variable=var,
                              command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=row_idx, column=0, padx=8, pady=6)

        if data["Image"]:
            img_lbl = ctk.CTkLabel(scrollable_frame, image=data["Image"], text="")
            img_lbl.image = data["Image"]
            img_lbl.grid(row=row_idx, column=1, padx=8, pady=6)
        else:
            img_lbl = ctk.CTkLabel(scrollable_frame, text="No Image", fg_color="#ddd", width=120, height=120)
            img_lbl.grid(row=row_idx, column=1, padx=8, pady=6)

        ctk.CTkLabel(scrollable_frame, text=data["Name"]).grid(row=row_idx, column=2, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row_idx, column=3, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=data["Modality"]).grid(row=row_idx, column=4, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=f"{data['CTDIvol']:.2f}").grid(row=row_idx, column=5, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=f"{data['DLP']:.2f}").grid(row=row_idx, column=6, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=f"{data['mSv']:.2f}").grid(row=row_idx, column=7, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=data["Sex"]).grid(row=row_idx, column=8, padx=8, pady=6)
        ctk.CTkLabel(scrollable_frame, text=data["DOB"]).grid(row=row_idx, column=9, padx=8, pady=6)


def on_check(data, var):
    if var.get():
        if data in selected_cases:
            return
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
            var.set(False)
            return
        selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def delete_selected_cases():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    for case in selected_cases[:]:
        if case in all_data:
            all_data.remove(case)
        selected_cases.remove(case)
    display_text_data()


def show_hl7_messages():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in selected_cases:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=15, pady=15)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Modality: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']:.2f} mGy\n"
            f"📏 DLP: {data['DLP']:.2f} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=13)).pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Elegant Display")
root.geometry("1300x900")

bg_img = Image.open("g.jpg").resize((1300, 900))
bg_img_tk = ImageTk.PhotoImage(bg_img)
background_label = ctk.CTkLabel(root, image=bg_img_tk)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files,
                              font=ctk.CTkFont(size=16, weight="bold"))
select_button.pack(pady=12)

sort_var = ctk.StringVar(value="Name")
sort_frame = ctk.CTkFrame(root)
sort_frame.pack(pady=6)
ctk.CTkLabel(sort_frame, text="Sort by:", font=ctk.CTkFont(size=14)).pack(side="left", padx=8)
ctk.CTkOptionMenu(sort_frame, variable=sort_var, values=["Name", "Date"], command=lambda e: display_text_data()).pack(side="left")

hl7_btn = ctk.CTkButton(root, text="Show HL7 Messages", command=show_hl7_messages,
                        font=ctk.CTkFont(size=14, weight="bold"))
hl7_btn.pack(pady=6)

content_container = ctk.CTkFrame(root, fg_color="#f8f9fa", corner_radius=15)
content_container.pack(expand=True, fill="both", padx=12, pady=12)

scrollable_frame = ctk.CTkScrollableFrame(content_container, corner_radius=10, fg_color="#ffffff")
scrollable_frame.pack(expand=True, fill="both", padx=10, pady=10)

btn_frame = ctk.CTkFrame(root)
btn_frame.pack(pady=10)
ctk.CTkButton(btn_frame, text="Delete Selected", command=delete_selected_cases).pack(side="left", padx=12)
ctk.CTkButton(btn_frame, text="View Selected Cases", command=show_selected_cases).pack(side="left", padx=12)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), modality)
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "Modality": modality,
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    header_font = ctk.CTkFont(size=14, weight="bold")
    row_colors = ["#eaf4fc", "#fdfdfd"]

    columns = ["Name", "Date", "Modality", "CTDIvol (mGy)", "DLP (mGy·cm)", "Dose (mSv)", "Sex", "DOB"]

    for col_idx, col_name in enumerate(columns):
        lbl = ctk.CTkLabel(scrollable_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=8, pady=6)

    for row_idx, data in enumerate(sorted_data, start=1):
        color = row_colors[row_idx % 2]
        for col_idx, col_name in enumerate(columns):
            value = data[col_name.split()[0] if '(' not in col_name else col_name.split()[0]]
            if isinstance(value, datetime):
                value = value.strftime("%Y-%m-%d")
            elif isinstance(value, float):
                value = f"{value:.2f}"
            ctk.CTkLabel(scrollable_frame, text=value, fg_color=color).grid(row=row_idx, column=col_idx, padx=6, pady=4, sticky="nsew")


def show_hl7_messages():
    if not all_data:
        messagebox.showinfo("Info", "No cases available.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in all_data:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


root = ctk.CTk()
root.title("DICOM Viewer - Simple Table")
root.geometry("1200x800")

header = ctk.CTkLabel(root, text="Click here to load DICOM files", font=ctk.CTkFont(size=24, weight="bold"))
header.pack(pady=20)
header.configure(text_color="red")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files,
                              font=ctk.CTkFont(size=16, weight="bold"))
select_button.pack(pady=10)

sort_var = ctk.StringVar(value="Name")
sort_frame = ctk.CTkFrame(root)
sort_frame.pack(pady=6)
ctk.CTkLabel(sort_frame, text="Sort by:", font=ctk.CTkFont(size=14)).pack(side="left", padx=8)
ctk.CTkOptionMenu(sort_frame, variable=sort_var, values=["Name", "Date"], command=lambda e: display_text_data()).pack(side="left")

hl7_btn = ctk.CTkButton(root, text="Show HL7 Messages", command=show_hl7_messages,
                        font=ctk.CTkFont(size=14, weight="bold"))
hl7_btn.pack(pady=6)

content_container = ctk.CTkFrame(root, corner_radius=15)
content_container.pack(expand=True, fill="both", padx=12, pady=12)

scrollable_frame = ctk.CTkScrollableFrame(content_container, corner_radius=10)
scrollable_frame.pack(expand=True, fill="both", padx=10, pady=10)

root.mainloop()


In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from datetime import datetime
import customtkinter as ctk
from PIL import Image, ImageTk
import csv
from cryptography.fernet import Fernet

# إعداد الواجهة
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("DICOM Viewer & HL7")
app.geometry("1400x800")

frame_top = ctk.CTkFrame(app)
frame_top.pack(pady=10)

frame_table = ctk.CTkFrame(app)
frame_table.pack(fill="both", expand=True, padx=20, pady=10)

scroll_canvas = tk.Canvas(frame_table)
scroll_canvas.pack(side="left", fill="both", expand=True)

scrollbar = ctk.CTkScrollbar(frame_table, command=scroll_canvas.yview)
scrollbar.pack(side="right", fill="y")

scroll_canvas.configure(yscrollcommand=scrollbar.set)
scroll_canvas.bind('<Configure>', lambda e: scroll_canvas.configure(scrollregion=scroll_canvas.bbox("all")))

scrollable_frame = ctk.CTkFrame(scroll_canvas)
scroll_canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

# البيانات
all_data = []
selected_cases = []

# المفاتيح للتشفير
key = Fernet.generate_key()
cipher = Fernet(key)

# تحميل ملفات DICOM

def load_dicom_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    for path in file_paths:
        dcm = pydicom.dcmread(path)
        data = {
            "Name": getattr(dcm, "PatientName", "N/A"),
            "Date": datetime.strptime(getattr(dcm, "StudyDate", "20000101"), "%Y%m%d"),
            "Modality": getattr(dcm, "Modality", "N/A"),
            "CTDIvol": float(getattr(dcm, "CTDIvol", 0)),
            "DLP": float(getattr(dcm, "DLP", 0)),
            "mSv": float(getattr(dcm, "DLP", 0)) * 0.015,
            "Sex": getattr(dcm, "PatientSex", "N/A"),
            "DOB": getattr(dcm, "PatientBirthDate", "N/A"),
            "Path": path
        }
        all_data.append(data)
    display_text_data()

# عرض البيانات بدون صور

def display_text_data():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    columns = {
        "Select": [],
        "Name": [],
        "Date": [],
        "Modality": [],
        "CTDIvol (mGy)": [],
        "DLP (mGy·cm)": [],
        "Dose (mSv)": [],
        "Sex": [],
        "DOB": []
    }

    header_font = ctk.CTkFont(size=14, weight="bold")
    for col_idx, col_name in enumerate(columns.keys()):
        lbl = ctk.CTkLabel(scrollable_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=6, pady=6, sticky="nsew")

    row_colors = ["#f0f8ff", "#e6f2ff"]

    for row_idx, data in enumerate(sorted_data, start=1):
        bg_color = row_colors[row_idx % 2]

        var = ctk.BooleanVar(value=(data in selected_cases))
        chk = ctk.CTkCheckBox(scrollable_frame, variable=var, text="", command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=row_idx, column=0, padx=6, pady=6)
        chk.configure(fg_color=bg_color, hover_color="#d1eaff")

        values = [
            data["Name"],
            data["Date"].strftime("%Y-%m-%d"),
            data["Modality"],
            f"{data['CTDIvol']:.2f}",
            f"{data['DLP']:.2f}",
            f"{data['mSv']:.2f}",
            data["Sex"],
            data["DOB"]
        ]

        for col_idx, val in enumerate(values, start=1):
            lbl = ctk.CTkLabel(scrollable_frame, text=val, fg_color=bg_color, anchor="w")
            lbl.grid(row=row_idx, column=col_idx, padx=6, pady=6, sticky="nsew")

# الترتيب والاختيار

def on_check(data, var):
    if var.get():
        selected_cases.append(data)
    else:
        selected_cases.remove(data)

# توليد رسالة HL7

def generate_hl7():
    if not selected_cases:
        return
    password = tk.simpledialog.askstring("Password", "Enter password to view HL7:", show='*')
    if password != "admin123":
        return

    for data in selected_cases:
        msg = f"MSH|^~\&|DICOM_APP|HOSPITAL|RAD_SYS|DEPT|{datetime.now().strftime('%Y%m%d%H%M')}||ORM^O01|{data['Date'].strftime('%Y%m%d')}|P|2.3\n"
        msg += f"PID|||{data['Name']}||{data['DOB']}|{data['Sex']}\n"
        msg += f"OBR|||1|{data['Modality']}|||{data['Date'].strftime('%Y%m%d')}\n"
        msg += f"OBX|||CTDIvol|{data['CTDIvol']}|mGy\n"
        msg += f"OBX|||DLP|{data['DLP']}|mGy*cm\n"
        msg += f"OBX|||Estimated Dose|{data['mSv']:.2f}|mSv\n"

        enc_msg = cipher.encrypt(msg.encode())
        top = tk.Toplevel()
        top.title("HL7 Message")
        txt = tk.Text(top, wrap="word", width=100, height=15)
        txt.pack(padx=10, pady=10)
        txt.insert("end", enc_msg.decode())

# تصدير إلى CSV

def export_to_csv():
    with open("rad.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Date", "Modality", "CTDIvol", "DLP", "Dose", "Sex", "DOB"])
        for data in all_data:
            writer.writerow([
                data["Name"],
                data["Date"].strftime("%Y-%m-%d"),
                data["Modality"],
                f"{data['CTDIvol']:.2f}",
                f"{data['DLP']:.2f}",
                f"{data['mSv']:.2f}",
                data["Sex"],
                data["DOB"]
            ])

# الواجهة العلوية

btn_load = ctk.CTkButton(frame_top, text="Load DICOM Files", command=load_dicom_files)
btn_load.pack(side="left", padx=10)

sort_var = tk.StringVar(value="Name")
sort_menu = ctk.CTkOptionMenu(frame_top, variable=sort_var, values=["Name", "Date"], command=lambda _: display_text_data())
sort_menu.pack(side="left", padx=10)

btn_hl7 = ctk.CTkButton(frame_top, text="Generate HL7 Message", command=generate_hl7)
btn_hl7.pack(side="left", padx=10)

btn_export = ctk.CTkButton(frame_top, text="Export to CSV", command=export_to_csv)
btn_export.pack(side="left", padx=10)

# عبارة البداية
placeholder_label = ctk.CTkLabel(app, text="Click here to load DICOM files", font=ctk.CTkFont(size=20, weight="bold"), text_color="blue")
placeholder_label.place(relx=0.5, rely=0.5, anchor="center")

app.mainloop()


In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from datetime import datetime
import customtkinter as ctk
from PIL import Image, ImageTk
import csv
from cryptography.fernet import Fernet

# إعداد الواجهة
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("DICOM Viewer & HL7")
app.geometry("1400x800")

frame_top = ctk.CTkFrame(app)
frame_top.pack(pady=10)

frame_table = ctk.CTkFrame(app)
frame_table.pack(fill="both", expand=True, padx=20, pady=10)

scroll_canvas = tk.Canvas(frame_table)
scroll_canvas.pack(side="left", fill="both", expand=True)

scrollbar = ctk.CTkScrollbar(frame_table, command=scroll_canvas.yview)
scrollbar.pack(side="right", fill="y")

scroll_canvas.configure(yscrollcommand=scrollbar.set)
scroll_canvas.bind('<Configure>', lambda e: scroll_canvas.configure(scrollregion=scroll_canvas.bbox("all")))

scrollable_frame = ctk.CTkFrame(scroll_canvas)
scroll_canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

# البيانات
all_data = []
selected_cases = []

# المفاتيح للتشفير
key = Fernet.generate_key()
cipher = Fernet(key)

# تحميل ملفات DICOM
def load_dicom_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not file_paths:
        return
    placeholder_label.place_forget()  # إخفاء عبارة البداية
    for path in file_paths:
        dcm = pydicom.dcmread(path)
        data = {
            "Name": str(getattr(dcm, "PatientName", "N/A")),
            "Date": datetime.strptime(getattr(dcm, "StudyDate", "20000101"), "%Y%m%d"),
            "Modality": str(getattr(dcm, "Modality", "N/A")),
            "CTDIvol": float(getattr(dcm, "CTDIvol", 0)),
            "DLP": float(getattr(dcm, "DLP", 0)),
            "mSv": float(getattr(dcm, "DLP", 0)) * 0.015,
            "Sex": str(getattr(dcm, "PatientSex", "N/A")),
            "DOB": str(getattr(dcm, "PatientBirthDate", "N/A")),
            "Path": path
        }
        all_data.append(data)
    display_text_data()

# عرض البيانات بدون صور
def display_text_data():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    columns = ["Select", "Name", "Date", "Modality", "CTDIvol (mGy)", "DLP (mGy·cm)", "Dose (mSv)", "Sex", "DOB"]

    header_font = ctk.CTkFont(size=16, weight="bold")
    for col_idx, col_name in enumerate(columns):
        lbl = ctk.CTkLabel(scrollable_frame, text=col_name, font=header_font, anchor="center", fg_color="#3B82F6", text_color="white", corner_radius=8)
        lbl.grid(row=0, column=col_idx, padx=6, pady=6, sticky="nsew")
        scrollable_frame.grid_columnconfigure(col_idx, weight=1)

    row_colors = ["#E3F2FD", "#BBDEFB"]

    for row_idx, data in enumerate(sorted_data, start=1):
        bg_color = row_colors[row_idx % 2]

        var = ctk.BooleanVar(value=(data in selected_cases))
        def cmd(d=data, v=var):
            on_check(d, v)
        chk = ctk.CTkCheckBox(scrollable_frame, variable=var, command=cmd)
        chk.grid(row=row_idx, column=0, padx=6, pady=6)
        chk.configure(fg_color=bg_color, hover_color="#90CAF9")

        values = [
            data["Name"],
            data["Date"].strftime("%Y-%m-%d"),
            data["Modality"],
            f"{data['CTDIvol']:.2f}",
            f"{data['DLP']:.2f}",
            f"{data['mSv']:.2f}",
            data["Sex"],
            data["DOB"]
        ]

        for col_idx, val in enumerate(values, start=1):
            lbl = ctk.CTkLabel(scrollable_frame, text=val, fg_color=bg_color, anchor="w", font=ctk.CTkFont(size=14))
            lbl.grid(row=row_idx, column=col_idx, padx=6, pady=6, sticky="nsew")

# الترتيب والاختيار
def on_check(data, var):
    if var.get():
        if data not in selected_cases:
            selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)

# حذف الحالات المحددة
def delete_selected():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    confirm = messagebox.askyesno("Confirm Delete", f"Delete {len(selected_cases)} selected cases?")
    if confirm:
        for data in selected_cases:
            if data in all_data:
                all_data.remove(data)
        selected_cases.clear()
        display_text_data()

# عرض الصور للحالات المحددة
def show_selected_images():
    count = len(selected_cases)
    if count not in [2, 4]:
        messagebox.showwarning("Warning", "Please select exactly 2 or 4 cases to display images.")
        return

    win = ctk.CTkToplevel(app)
    win.title(f"Showing {count} Selected DICOM Images")
    win.geometry("1000x700")

    # إعداد شبكة عرض الصور
    rows = 1 if count == 2 else 2
    cols = 2

    for idx, data in enumerate(selected_cases):
        # قراءة صورة DICOM وتحويلها إلى صورة PIL
        dcm = pydicom.dcmread(data["Path"])
        if 'PixelData' not in dcm:
            continue  # إذا ما فيه صورة

        # تحويل الصورة إلى PIL image (باستخدام windowing إذا موجود)
        arr = dcm.pixel_array
        image = Image.fromarray(arr)
        image = image.convert("L").resize((400, 400), Image.ANTIALIAS)

        img_tk = ImageTk.PhotoImage(image)
        lbl_img = ctk.CTkLabel(win, image=img_tk)
        lbl_img.image = img_tk  # ضروري للاحتفاظ بالصورة
        lbl_img.grid(row=idx//cols*2, column=idx%cols, padx=10, pady=10)

        # عرض بيانات مختصرة تحت الصورة
        info_text = f"Name: {data['Name']}\nDate: {data['Date'].strftime('%Y-%m-%d')}\nModality: {data['Modality']}\nDose: {data['mSv']:.2f} mSv"
        lbl_info = ctk.CTkLabel(win, text=info_text, justify="left", font=ctk.CTkFont(size=12))
        lbl_info.grid(row=(idx//cols)*2 +1, column=idx%cols, padx=10, pady=(0,10))

# توليد رسالة HL7 (نفس السابق)
def generate_hl7():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected for HL7 generation.")
        return
    password = simpledialog.askstring("Password", "Enter password to view HL7:", show='*')
    if password != "admin123":
        messagebox.showerror("Error", "Incorrect password.")
        return

    for data in selected_cases:
        msg = f"MSH|^~\\&|DICOM_APP|HOSPITAL|RAD_SYS|DEPT|{datetime.now().strftime('%Y%m%d%H%M')}||ORM^O01|{data['Date'].strftime('%Y%m%d')}|P|2.3\n"
        msg += f"PID|||{data['Name']}||{data['DOB']}|{data['Sex']}\n"
        msg += f"OBR|||1|{data['Modality']}|||{data['Date'].strftime('%Y%m%d')}\n"
        msg += f"OBX|||CTDIvol|{data['CTDIvol']}|mGy\n"
        msg += f"OBX|||DLP|{data['DLP']}|mGy*cm\n"
        msg += f"OBX|||Estimated Dose|{data['mSv']:.2f}|mSv\n"

        enc_msg = cipher.encrypt(msg.encode())
        top = tk.Toplevel()
        top.title("HL7 Message")
        txt = tk.Text(top, wrap="word", width=100, height=15)
        txt.pack(padx=10, pady=10)
        txt.insert("end", enc_msg.decode())

# تصدير إلى CSV (نفس السابق)
def export_to_csv():
    with open("rad.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Date", "Modality", "CTDIvol", "DLP", "Dose", "Sex", "DOB"])
        for data in all_data:
            writer.writerow([
                data["Name"],
                data["Date"].strftime("%Y-%m-%d"),
                data["Modality"],
                f"{data['CTDIvol']:.2f}",
                f"{data['DLP']:.2f}",
                f"{data['mSv']:.2f}",
                data["Sex"],
                data["DOB"]
            ])

# الواجهة العلوية مع الأزرار الجديدة
btn_load = ctk.CTkButton(frame_top, text="Load DICOM Files", command=load_dicom_files)
btn_load.pack(side="left", padx=10)

sort_var = tk.StringVar(value="Name")
sort_menu = ctk.CTkOptionMenu(frame_top, variable=sort_var, values=["Name", "Date"], command=lambda _: display_text_data())
sort_menu.pack(side="left", padx=10)

btn_hl7 = ctk.CTkButton(frame_top, text="Generate HL7 Message", command=generate_hl7)
btn_hl7.pack(side="left", padx=10)

btn_export = ctk.CTkButton(frame_top, text="Export to CSV", command=export_to_csv)
btn_export.pack(side="left", padx=10)

btn_delete = ctk.CTkButton(frame_top, text="Delete Selected", command=delete_selected, fg_color="red", hover_color="#ff6666")
btn_delete.pack(side="left", padx=10)

btn_show_images = ctk.CTkButton(frame_top, text="Show Selected Images (2 or 4)", command=show_selected_images)
btn_show_images.pack(side="left", padx=10)

# عبارة البداية
placeholder_label = ctk.CTkLabel(app, text="Click here to load DICOM files", font=ctk.CTkFont(size=20, weight="bold"), text_color="blue")
placeholder_label.place(relx=0.5, rely=0.5, anchor="center")

app.mainloop()


In [ ]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog, simpledialog
from datetime import datetime
import customtkinter as ctk
from PIL import Image, ImageTk
import csv
from cryptography.fernet import Fernet

# إعداد الواجهة
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("DICOM Viewer & HL7")
app.geometry("1400x800")

frame_top = ctk.CTkFrame(app)
frame_top.pack(pady=10)

frame_table = ctk.CTkFrame(app)
frame_table.pack(fill="both", expand=True, padx=20, pady=10)

scroll_canvas = tk.Canvas(frame_table)
scroll_canvas.pack(side="left", fill="both", expand=True)

scrollbar = ctk.CTkScrollbar(frame_table, command=scroll_canvas.yview)
scrollbar.pack(side="right", fill="y")

scroll_canvas.configure(yscrollcommand=scrollbar.set)
scroll_canvas.bind('<Configure>', lambda e: scroll_canvas.configure(scrollregion=scroll_canvas.bbox("all")))

scrollable_frame = ctk.CTkFrame(scroll_canvas)
scroll_canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

# البيانات
all_data = []
selected_cases = []

# المفاتيح للتشفير
key = Fernet.generate_key()
cipher = Fernet(key)

# تحميل ملفات DICOM

def load_dicom_files():
    global all_data, selected_cases
    file_paths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not file_paths:
        return
    all_data.clear()
    selected_cases.clear()
    for path in file_paths:
        try:
            dcm = pydicom.dcmread(path)
            data = {
                "Name": str(getattr(dcm, "PatientName", "N/A")),
                "Date": datetime.strptime(getattr(dcm, "StudyDate", "20000101"), "%Y%m%d"),
                "Modality": getattr(dcm, "Modality", "N/A"),
                "CTDIvol": float(getattr(dcm, "CTDIvol", 0)),
                "DLP": float(getattr(dcm, "DLP", 0)),
                "mSv": float(getattr(dcm, "DLP", 0)) * 0.015,
                "Sex": getattr(dcm, "PatientSex", "N/A"),
                "DOB": getattr(dcm, "PatientBirthDate", "N/A"),
                "Path": path
            }
            all_data.append(data)
        except Exception as e:
            print(f"Error reading {path}: {e}")
    placeholder_label.place_forget()
    display_text_data()

# عرض البيانات بدون صور

def display_text_data():
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    if not all_data:
        return

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    columns = {
        "Select": [],
        "Name": [],
        "Date": [],
        "Modality": [],
        "CTDIvol (mGy)": [],
        "DLP (mGy·cm)": [],
        "Dose (mSv)": [],
        "Sex": [],
        "DOB": []
    }

    header_font = ctk.CTkFont(size=14, weight="bold")
    for col_idx, col_name in enumerate(columns.keys()):
        lbl = ctk.CTkLabel(scrollable_frame, text=col_name, font=header_font, anchor="center", text_color="black")
        lbl.grid(row=0, column=col_idx, padx=6, pady=6, sticky="nsew")

    row_colors = ["#f0f8ff", "#e6f2ff"]

    for row_idx, data in enumerate(sorted_data, start=1):
        bg_color = row_colors[row_idx % 2]

        var = ctk.BooleanVar(value=(data in selected_cases))
        chk = ctk.CTkCheckBox(scrollable_frame, variable=var, text="", command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=row_idx, column=0, padx=6, pady=6)
        # ضبط خلفية checkbox
        chk.configure(fg_color="#0078d7", hover_color="#005a9e")

        values = [
            data["Name"],
            data["Date"].strftime("%Y-%m-%d"),
            data["Modality"],
            f"{data['CTDIvol']:.2f}",
            f"{data['DLP']:.2f}",
            f"{data['mSv']:.2f}",
            data["Sex"],
            data["DOB"]
        ]

        for col_idx, val in enumerate(values, start=1):
            lbl = ctk.CTkLabel(scrollable_frame, text=val, fg_color=bg_color, anchor="w", text_color="black")
            lbl.grid(row=row_idx, column=col_idx, padx=6, pady=6, sticky="nsew")

# الترتيب والاختيار

def on_check(data, var):
    if var.get():
        if data not in selected_cases:
            selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)

# توليد رسالة HL7

def generate_hl7():
    if not selected_cases:
        ctk.CTkMessageBox(title="Warning", message="Please select at least one case!")
        return
    password = simpledialog.askstring("Password", "Enter password to view HL7:", show='*')
    if password != "admin123":
        ctk.CTkMessageBox(title="Error", message="Incorrect password!")
        return

    for data in selected_cases:
        msg = f"MSH|^~\&|DICOM_APP|HOSPITAL|RAD_SYS|DEPT|{datetime.now().strftime('%Y%m%d%H%M')}||ORM^O01|{data['Date'].strftime('%Y%m%d')}|P|2.3\n"
        msg += f"PID|||{data['Name']}||{data['DOB']}|{data['Sex']}\n"
        msg += f"OBR|||1|{data['Modality']}|||{data['Date'].strftime('%Y%m%d')}\n"
        msg += f"OBX|||CTDIvol|{data['CTDIvol']}|mGy\n"
        msg += f"OBX|||DLP|{data['DLP']}|mGy*cm\n"
        msg += f"OBX|||Estimated Dose|{data['mSv']:.2f}|mSv\n"

        enc_msg = cipher.encrypt(msg.encode())
        top = tk.Toplevel()
        top.title("HL7 Message")
        txt = tk.Text(top, wrap="word", width=100, height=15)
        txt.pack(padx=10, pady=10)
        txt.insert("end", enc_msg.decode())

# تصدير إلى CSV

def export_to_csv():
    if not all_data:
        ctk.CTkMessageBox(title="Warning", message="No data to export!")
        return
    with open("rad.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Name", "Date", "Modality", "CTDIvol", "DLP", "Dose", "Sex", "DOB"])
        for data in all_data:
            writer.writerow([
                data["Name"],
                data["Date"].strftime("%Y-%m-%d"),
                data["Modality"],
                f"{data['CTDIvol']:.2f}",
                f"{data['DLP']:.2f}",
                f"{data['mSv']:.2f}",
                data["Sex"],
                data["DOB"]
            ])
    ctk.CTkMessageBox(title="Success", message="Data exported to rad.csv")

# عرض الصور للحالات المختارة (2 أو 4)

def show_selected_images():
    if len(selected_cases) not in [2, 4]:
        ctk.CTkMessageBox(title="Warning", message="Please select exactly 2 or 4 cases to view images.")
        return

    win = ctk.CTkToplevel(app)
    win.geometry("1000x700")
    win.title("Selected DICOM Images")

    canvas = tk.Canvas(win)
    canvas.pack(fill="both", expand=True)

    frame = ctk.CTkFrame(canvas)
    canvas.create_window((0,0), window=frame, anchor="nw")

    images = []  # لتخزين الصور وعدم مسحها من الذاكرة

    def load_image(path):
        try:
            dcm = pydicom.dcmread(path)
            pixel_array = dcm.pixel_array
            img = Image.fromarray(pixel_array)
            img = img.convert("L").resize((400, 400))
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            return None

    rows = 2 if len(selected_cases) == 4 else 1
    cols = 2 if len(selected_cases) >= 2 else 1

    for idx, data in enumerate(selected_cases):
        img = load_image(data["Path"])
        if img is None:
            continue
        images.append(img)  # لمنع جمع القمامة
        img_label = ctk.CTkLabel(frame, image=img, text="")
        img_label.grid(row=idx//cols*2, column=idx%cols, padx=10, pady=10)

        info = (f"Name: {data['Name']}\n"
                f"Date: {data['Date'].strftime('%Y-%m-%d')}\n"
                f"Modality: {data['Modality']}\n"
                f"CTDIvol: {data['CTDIvol']:.2f} mGy\n"
                f"DLP: {data['DLP']:.2f} mGy·cm\n"
                f"Dose: {data['mSv']:.2f} mSv\n"
                f"Sex: {data['Sex']}\n"
                f"DOB: {data['DOB']}")

        info_label = ctk.CTkLabel(frame, text=info, justify="left", anchor="w")
        info_label.grid(row=idx//cols*2+1, column=idx%cols, padx=10, pady=5, sticky="w")

    def on_frame_configure(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    frame.bind("<Configure>", on_frame_configure)

# حذف الحالات المختارة

def delete_selected_cases():
    global all_data, selected_cases
    if not selected_cases:
        ctk.CTkMessageBox(title="Warning", message="No cases selected to delete!")
        return
    confirm = ctk.CTkMessageBox(title="Confirm Delete", message="Are you sure you want to delete selected cases?", icon="warning", yes_no=True)
    if confirm == "yes":
        for data in selected_cases:
            if data in all_data:
                all_data.remove(data)
        selected_cases.clear()
        display_text_data()

# تفعيل العبارة التي تُنقر للتحميل

def on_placeholder_click(event):
    load_dicom_files()

# الواجهة العلوية

btn_load = ctk.CTkButton(frame_top, text="Load DICOM Files", command=load_dicom_files)
btn_load.pack(side="left", padx=10)

sort_var = tk.StringVar(value="Name")
sort_menu = ctk.CTkOptionMenu(frame_top, variable=sort_var, values=["Name", "Date"], command=lambda _: display_text_data())
sort_menu.pack(side="left", padx=10)

btn_hl7 = ctk.CTkButton(frame_top, text="Generate HL7 Message", command=generate_hl7)
btn_hl7.pack(side="left", padx=10)

btn_export = ctk.CTkButton(frame_top, text="Export to CSV", command=export_to_csv)
btn_export.pack(side="left", padx=10)

btn_show_images = ctk.CTkButton(frame_top, text="Show Selected Images (2 or 4)", command=show_selected_images)
btn_show_images.pack(side="left", padx=10)

btn_delete = ctk.CTkButton(frame_top, text="Delete Selected Cases", command=delete_selected_cases)
btn_delete.pack(side="left", padx=10)

# عبارة البداية مفعلة

placeholder_label = ctk.CTkLabel(app, text="Click here to load DICOM files", font=ctk.CTkFont(size=20, weight="bold"), text_color="blue", cursor="hand2")
placeholder_label.place(relx=0.5, rely=0.5, anchor="center")
placeholder_label.bind("<Button-1>", on_placeholder_click)

app.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []  # مهم جداً لتخزين الصور حتى لا يتم جمعها من الذاكرة


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    global image_refs
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    image_refs.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            # لن نخزن الصور هنا حتى لا تظهر في الجدول

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Image": None,  # لن نعرض الصور في الجدول
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    columns = [
        "Select",
        "Name",
        "Date",
        "Modality",
        "CTDIvol (mGy)",
        "DLP (mGy·cm)",
        "Dose (mSv)",
        "Sex",
        "DOB"
    ]

    header_font = ctk.CTkFont(size=14, weight="bold")
    for col_idx, col_name in enumerate(columns):
        lbl = ctk.CTkLabel(content_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=8, pady=5)

    for row_idx, data in enumerate(sorted_data, start=1):
        bg_color = "#ffffff" if row_idx % 2 == 1 else "#f0f0f0"

        var = ctk.BooleanVar(value=(data in selected_cases))

        chk = ctk.CTkCheckBox(content_frame, variable=var,
                              command=lambda d=data, v=var: on_check(d, v),
                              fg_color="#0078d7")
        chk.grid(row=row_idx, column=0, padx=8, pady=6)
        chk.configure(bg_color=bg_color)

        labels_text = [
            data["Name"],
            data["Date"].strftime("%Y-%m-%d"),
            data["Modality"],
            f"{data['CTDIvol']:.2f}",
            f"{data['DLP']:.2f}",
            f"{data['mSv']:.2f}",
            data["Sex"],
            data["DOB"]
        ]

        for col_idx, text in enumerate(labels_text, start=1):
            lbl = ctk.CTkLabel(content_frame, text=text, anchor="w", bg_color=bg_color, corner_radius=5, padx=6, pady=2)
            lbl.grid(row=row_idx, column=col_idx, sticky="ew", padx=4, pady=2)

    for i in range(len(columns)):
        content_frame.grid_columnconfigure(i, weight=1)


def on_check(data, var):
    if var.get():
        if data in selected_cases:
            return
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
            var.set(False)
            return
        selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def delete_selected_cases():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    for case in selected_cases[:]:
        if case in all_data:
            all_data.remove(case)
        selected_cases.remove(case)
    display_text_data()


def show_hl7_messages():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in selected_cases:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=15, pady=15)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"📅 Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"🖥 Modality: {data['Modality']}\n"
            f"💉 CTDIvol: {data['CTDIvol']:.2f} mGy\n"
            f"📊 DLP: {data['DLP']:.2f} mGy·cm\n"
            f"⚠ Dose: {data['mSv']:.2f} mSv\n"
            f"👩‍⚕ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}\n"
        )
        lbl_info = ctk.CTkLabel(frame, text=info, justify="left")
        lbl_info.pack(padx=10, pady=10)


app = ctk.CTk()
app.title("DICOM Viewer without Images in Table")
app.geometry("1150x650")

header_frame = ctk.CTkFrame(app)
header_frame.pack(fill="x", pady=10)

load_btn = ctk.CTkButton(header_frame, text="Load DICOM Files", command=read_dicom_files)
load_btn.pack(side="left", padx=10)

sort_var = ctk.StringVar(value="Name")
sort_menu = ctk.CTkOptionMenu(header_frame, values=["Name", "Date", "CTDIvol", "Modality"], variable=sort_var,
                              command=lambda _: display_text_data())
sort_menu.pack(side="left", padx=10)

show_btn = ctk.CTkButton(header_frame, text="Show Selected (2 or 4)", command=show_selected_cases)
show_btn.pack(side="left", padx=10)

hl7_btn = ctk.CTkButton(header_frame, text="Show HL7 Messages", command=show_hl7_messages)
hl7_btn.pack(side="left", padx=10)

delete_btn = ctk.CTkButton(header_frame, text="Delete Selected", command=delete_selected_cases)
delete_btn.pack(side="left", padx=10)

content_frame = ctk.CTkScrollableFrame(app)
content_frame.pack(expand=True, fill="both", padx=15, pady=10)

app.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []  # لتخزين الصور فقط عند عرض الحالات المختارة


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    global image_refs
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    image_refs.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    header_font = ctk.CTkFont(size=14, weight="bold")

    headers = ["Select", "Name", "Date", "Modality", "CTDIvol (mGy)", "DLP (mGy·cm)", "Dose (mSv)", "Sex", "DOB"]
    for col_idx, col_name in enumerate(headers):
        lbl = ctk.CTkLabel(content_frame, text=col_name, font=header_font, anchor="center")
        lbl.grid(row=0, column=col_idx, padx=10, pady=8)

    for row_idx, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=(data in selected_cases))

        chk = ctk.CTkCheckBox(content_frame, variable=var,
                              command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=row_idx, column=0, padx=10, pady=8)

        ctk.CTkLabel(content_frame, text=data["Name"], anchor="w").grid(row=row_idx, column=1, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row_idx, column=2, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=data["Modality"]).grid(row=row_idx, column=3, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']:.2f}").grid(row=row_idx, column=4, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=f"{data['DLP']:.2f}").grid(row=row_idx, column=5, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row_idx, column=6, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row_idx, column=7, padx=10, pady=8)
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row_idx, column=8, padx=10, pady=8)


def on_check(data, var):
    if var.get():
        if data in selected_cases:
            return
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
            var.set(False)
            return
        selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def delete_selected_cases():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to delete.")
        return
    for case in selected_cases[:]:
        if case in all_data:
            all_data.remove(case)
        selected_cases.remove(case)
    display_text_data()


def show_hl7_messages():
    if not selected_cases:
        messagebox.showinfo("Info", "No cases selected to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    combined_msg = ""
    for data in selected_cases:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['Modality']}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            combined_msg += f"--- HL7 Message for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']}) ---\n"
            combined_msg += hl7 + "\n\n"
        else:
            combined_msg += f"HL7 message not found for {data['Name']} on {data['Date'].strftime('%Y-%m-%d')} ({data['Modality']})\n\n"

    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")

    textbox = ctk.CTkTextbox(top, wrap="word")
    textbox.pack(expand=True, fill="both", padx=10, pady=10)
    textbox.insert("0.0", combined_msg)
    textbox.configure(state="disabled")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    global image_refs
    image_refs.clear()

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=15, pady=15)

        # نجهز الصورة عند العرض فقط
        img = None
        try:
            ds = data["Ds"]
            arr = ds.pixel_array
            from PIL import Image, ImageTk

            if len(arr.shape) == 3 and arr.shape[0] == 3:
                arr = arr.transpose(1, 2, 0)
            pil_img = Image.fromarray(arr)
            pil_img.thumbnail((250, 250))
            img = ImageTk.PhotoImage(pil_img)
            image_refs.append(img)
        except Exception:
            pass

        if img:
            img_label = ctk.CTkLabel(frame, image=img, text="")
            img_label.image = img
            img_label.pack(pady=10)
        else:
            ctk.CTkLabel(frame, text="No Image", fg_color="#ddd", width=250, height=250).pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Modality: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']:.2f} mGy\n"
            f"📏 DLP: {data['DLP']:.2f} mGy·cm\n"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(padx=10, pady=10)


root = ctk.CTk()
root.geometry("1200x650")
root.title("DICOM Cases Manager")

# إضافة صورة خلفية
bg_image = ctk.CTkImage(light_image=ctk.CTkImage.Image.open("background.jpg"),
                        dark_image=ctk.CTkImage.Image.open("background.jpg"), size=(1200, 650))
background_label = ctk.CTkLabel(root, image=bg_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

header = ctk.CTkLabel(root, text="Click 'Load DICOM Files' to import files", font=ctk.CTkFont(size=18, weight="bold"))
header.pack(pady=10)

load_btn = ctk.CTkButton(root, text="Load DICOM Files", command=read_dicom_files)
load_btn.pack(pady=10)

sort_var = ctk.StringVar(value="Name")
sort_frame = ctk.CTkFrame(root)
sort_frame.pack(pady=6)
ctk.CTkLabel(sort_frame, text="Sort by:", font=ctk.CTkFont(size=14)).pack(side="left", padx=8)
ctk.CTkOptionMenu(sort_frame, variable=sort_var, values=["Name", "Date"], command=lambda e: display_text_data()).pack(side="left")

hl7_btn = ctk.CTkButton(root, text="Show HL7 Messages", command=show_hl7_messages,
                        font=ctk.CTkFont(size=14, weight="bold"))
hl7_btn.pack(pady=6)

content_frame = ctk.CTkFrame(root, fg_color="#f8f9fa", corner_radius=15)
content_frame.pack(expand=True, fill="both", padx=12, pady=12)

btn_frame = ctk.CTkFrame(root)
btn_frame.pack(pady=10)
ctk.CTkButton(btn_frame, text="Delete Selected", command=delete_selected_cases).pack(side="left", padx=12)
ctk.CTkButton(btn_frame, text="View Selected Cases", command=show_selected_cases).pack(side="left", padx=12)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    # تجميع الحالات حسب (Name, Date, StudyID) لتجنب التكرار للحالة نفسها في نفس التاريخ
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            # إذا موجودش المفتاح نضيف الحالة
            if key not in temp_cases:
                img = None
                if 'PixelData' in ds:
                    img_pil = Image.fromarray(ds.pixel_array)
                    img_pil.thumbnail((200, 200))
                    img = ImageTk.PhotoImage(img_pil)

                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path,
                    "Modality": getattr(ds, "Modality", "Unknown"),
                    "Dataset": ds  # حفظ الـ dataset لرسالة HL7 لاحقا
                }
                temp_cases[key] = data_dict

                # حفظ رسالة HL7
                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{data_dict['StudyID']}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())
    display_text_data()

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option] if sort_option != "Name" else x["Name"].lower())

    # إنشاء إطار للجدول مع لون خلفية أبيض بدون شفافية لتجنب الخطأ
    table_frame = ctk.CTkFrame(content_frame, fg_color="#ffffff", corner_radius=10)
    table_frame.pack(fill="both", expand=True, padx=15, pady=10)

    # رؤوس الجدول
    headers = ["Select", "Name", "Date", "Modality", "Study ID", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)"]
    for col, header in enumerate(headers):
        label = ctk.CTkLabel(table_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5)

    # متغيرات تحكم checkboxes
    check_vars.clear()

    for row, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=data in selected_cases)
        check_vars.append((var, data))

        chk = ctk.CTkCheckBox(table_frame, variable=var)
        chk.grid(row=row, column=0, padx=5, pady=5)

        ctk.CTkLabel(table_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=2, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=data["Modality"]).grid(row=row, column=3, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=str(data["StudyID"])).grid(row=row, column=4, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=5, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=str(data["CTDIvol"])).grid(row=row, column=6, padx=5, pady=5)
        ctk.CTkLabel(table_frame, text=str(data["DLP"])).grid(row=row, column=7, padx=5, pady=5)

def update_selected_cases():
    selected_cases.clear()
    for var, data in check_vars:
        if var.get():
            selected_cases.append(data)

def show_hl7_message():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    # عرض رسالة HL7 لآخر حالة محددة (يمكن تعديلها لتحديد أي حالة)
    data = selected_cases[-1]
    hl7_filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}.hl7"
    if os.path.exists(hl7_filename):
        with open(hl7_filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def show_selected_cases():
    update_selected_cases()
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()  # عرض النافذة أمام النوافذ الأخرى

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

# متغيرات global للتحكم في checkboxes
check_vars = []

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

hl7_button = ctk.CTkButton(root, text="📨 Show HL7 Message", command=show_hl7_message)
hl7_button.place(relx=0.2, rely=0.03)

sort_var = ctk.StringVar(value="Name")
sort_label = ctk.CTkLabel(root, text="Sort by:")
sort_label.place(relx=0.4, rely=0.035)
sort_option = ctk.CTkOptionMenu(root, variable=sort_var, values=["Name", "Date"])
sort_option.place(relx=0.48, rely=0.03)

show_button = ctk.CTkButton(root, text="Show Selected Cases", command=show_selected_cases)
show_button.place(relx=0.58, rely=0.03)

content_frame = ctk.CTkFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("dark-blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
check_vars = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in temp_cases:
                img = None
                if 'PixelData' in ds:
                    img_pil = Image.fromarray(ds.pixel_array)
                    img_pil.thumbnail((200, 200))
                    img = ImageTk.PhotoImage(img_pil)

                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path,
                    "Modality": getattr(ds, "Modality", "Unknown"),
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{data_dict['StudyID']}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())
    display_text_data()

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option] if sort_option != "Name" else x["Name"].lower())

    # جدول داخل Scrollable Frame
    scroll_frame = ctk.CTkScrollableFrame(content_frame, corner_radius=10, fg_color="#ffffff")
    scroll_frame.pack(fill="both", expand=True, padx=20, pady=10)

    headers = ["Select", "Name", "Date", "Modality", "Study ID", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)"]
    for col, header in enumerate(headers):
        lbl = ctk.CTkLabel(scroll_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        lbl.grid(row=0, column=col, padx=10, pady=10, sticky="w")

    check_vars.clear()
    for row, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=data in selected_cases)
        check_vars.append((var, data))
        chk = ctk.CTkCheckBox(scroll_frame, variable=var)
        chk.grid(row=row, column=0, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(scroll_frame, text=data["Name"]).grid(row=row, column=1, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=2, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=data["Modality"]).grid(row=row, column=3, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["StudyID"])).grid(row=row, column=4, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=5, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["CTDIvol"])).grid(row=row, column=6, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["DLP"])).grid(row=row, column=7, padx=10, pady=5, sticky="w")

def update_selected_cases():
    selected_cases.clear()
    for var, data in check_vars:
        if var.get():
            selected_cases.append(data)

def show_hl7_message():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    data = selected_cases[-1]
    hl7_filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}.hl7"
    if os.path.exists(hl7_filename):
        with open(hl7_filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def show_selected_cases():
    update_selected_cases()
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, fg_color="#f0f0f0", corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=10)

    # جعل الشبكة تتوسع بشكل متساو
    for i in range(2):
        top.grid_columnconfigure(i, weight=1)
        top.grid_rowconfigure(i, weight=1)

def resize_bg(event):
    global bg_img_resized, bg_label
    new_width = event.width
    new_height = event.height
    resized = bg_img_orig.resize((new_width, new_height), Image.ANTIALIAS)
    bg_img_resized = ImageTk.PhotoImage(resized)
    bg_label.configure(image=bg_img_resized)

root = ctk.CTk()
root.title("DICOM Viewer - Responsive Design")
root.geometry("1300x900")

# تحميل صورة الخلفية
bg_img_orig = Image.open("background.jpg")  # ضع مسار صورتك هنا
bg_img_resized = ImageTk.PhotoImage(bg_img_orig)
bg_label = ctk.CTkLabel(root, image=bg_img_resized, text="")
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
root.bind("<Configure>", resize_bg)

# زر اختيار ملفات DICOM
load_button = ctk.CTkButton(root, text="Load DICOM Files", command=read_dicom_files)
load_button.place(relx=0.02, rely=0.02, width=150, height=35)

# زر عرض HL7
hl7_button = ctk.CTkButton(root, text="Show HL7 Message", command=show_hl7_message)
hl7_button.place(relx=0.20, rely=0.02, width=150, height=35)

# اختيار الترتيب
sort_var = ctk.StringVar(value="Name")
sort_label = ctk.CTkLabel(root, text="Sort by:")
sort_label.place(relx=0.38, rely=0.03)
sort_option = ctk.CTkOptionMenu(root, variable=sort_var, values=["Name", "Date"])
sort_option.place(relx=0.46, rely=0.02, width=120, height=30)

# زر عرض الحالات المختارة
show_button = ctk.CTkButton(root, text="Show Selected Cases", command=show_selected_cases)
show_button.place(relx=0.62, rely=0.02, width=180, height=35)

# إطار المحتوى للجدول
content_frame = ctk.CTkFrame(root, fg_color="#ffffffcc", corner_radius=15)
content_frame.place(relx=0.02, rely=0.07, relwidth=0.96, relheight=0.90)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# Convert DICOM data to HL7 message
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    unique_keys = set()
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            modality = getattr(ds, "Modality", "Unknown")
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            # Use unique key to avoid duplicate by Name+Date+Modality+StudyID+Accession
            study_id = getattr(ds, "StudyID", "")
            accession = getattr(ds, "AccessionNumber", "")
            unique_key = (name, date_obj, modality, study_id, accession)
            if unique_key in unique_keys:
                continue  # skip duplicate
            unique_keys.add(unique_key)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": study_id,
                "Accession": accession,
                "Modality": modality,
                "Path": path,
                "Image": None  # will load image on demand later
            }

            all_data.append(data_dict)

            # Save HL7 message
            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{study_id}_{accession}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table_data()

def display_table_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # Headers
    headers = ["Select", "Name", "Date", "Modality", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB", "StudyID", "Accession"]
    for col, text in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=text, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="nsew")

    # Sorting data
    sort_key = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_key] if sort_key != "Date" else x[sort_key])

    # Fill rows
    for row, data in enumerate(sorted_data, start=1):
        # Checkbox for selection
        var = ctk.BooleanVar(value=(data in selected_cases))
        cb = ctk.CTkCheckBox(content_frame, variable=var,
                             command=lambda d=data, v=var: on_select_checkbox(d, v))
        cb.grid(row=row, column=0, padx=5, pady=2)

        ctk.CTkLabel(content_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=2, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["Modality"]).grid(row=row, column=3, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=4, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']:.2f}").grid(row=row, column=5, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=f"{data['DLP']:.2f}").grid(row=row, column=6, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row, column=7, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row, column=8, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["StudyID"]).grid(row=row, column=9, padx=5, pady=2)
        ctk.CTkLabel(content_frame, text=data["Accession"]).grid(row=row, column=10, padx=5, pady=2)

def on_select_checkbox(data, var):
    if var.get():
        if data not in selected_cases:
            if len(selected_cases) >= 4:
                messagebox.showwarning("Limit Reached", "You can select up to 4 cases only.")
                var.set(False)
                return
            selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)

def show_hl7_message():
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    hl7_text = ""
    for data in selected_cases:
        name = data["Name"]
        date_str = data["Date"].strftime("%Y%m%d")
        study_id = data["StudyID"]
        accession = data["Accession"]
        filename = f"{HL7_DIR}/{name}_{date_str}_{study_id}_{accession}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7_text += f"--- HL7 for {name} on {date_str} ---\n"
                hl7_text += f.read() + "\n\n"
        else:
            hl7_text += f"HL7 message not found for {name} on {date_str}\n\n"

    # Show in scrollable window
    top = ctk.CTkToplevel()
    top.title("HL7 Messages")
    top.geometry("700x500")
    text_box = ctk.CTkTextbox(top, wrap="word")
    text_box.pack(expand=True, fill="both", padx=10, pady=10)
    text_box.insert("0.0", hl7_text)
    text_box.configure(state="disabled")

def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases to display images.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()  # bring window to front

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10, border_width=1)
        frame.grid(row=row, column=col, padx=15, pady=15, sticky="nsew")

        # Load image if not loaded yet
        if data["Image"] is None:
            try:
                ds = pydicom.dcmread(data["Path"])
                if "PixelData" in ds:
                    pil_img = Image.fromarray(ds.pixel_array)
                    pil_img.thumbnail((400, 400))
                    data["Image"] = ctk.CTkImage(pil_img)
            except:
                data["Image"] = None

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"Name: {data['Name']}\n"
            f"ID: {data['StudyID']}\n"
            f"Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"Modality: {data['Modality']}\n"
            f"Dose: {data['mSv']:.2f} mSv\n"
            f"CTDIvol: {data['CTDIvol']:.2f} mGy\n"
            f"DLP: {data['DLP']:.2f} mGy·cm\n"
            f"Sex: {data['Sex']}\n"
            f"DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

    # Make grid cells expand
    for i in range(2):
        top.grid_columnconfigure(i, weight=1)
        top.grid_rowconfigure(i, weight=1)

# Main Window
root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

# Background Image
bg_pil_image = Image.open("g.jpg")
bg_image = ctk.CTkImage(light_image=bg_pil_image, dark_image=bg_pil_image, size=(1200, 850))
background_label = ctk.CTkLabel(root, image=bg_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Buttons
select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

hl7_button = ctk.CTkButton(root, text="📨 Show HL7 Messages", command=show_hl7_message)
hl7_button.place(relx=0.42, rely=0.03)

# Sorting Option
sort_var = ctk.StringVar(value="Name")
sort_label = ctk.CTkLabel(root, text="Sort by:")
sort_label.place(relx=0.62, rely=0.03)
sort_option = ctk.CTkOptionMenu(root, values=["Name", "Date", "Modality", "mSv"], variable=sort_var, command=lambda e: display_table_data())
sort_option.place(relx=0.70, rely=0.03)

# Scrollable Frame for Table
table_frame = ctk.CTkFrame(root, fg_color="#ffffffaa", corner_radius=10)
table_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

canvas = ctk.CTkCanvas(table_frame, bg="#ffffff00", highlightthickness=0)
scrollbar = ctk.CTkScrollbar(table_frame, orientation="vertical", command=canvas.yview)
scrollable_frame = ctk.CTkFrame(canvas)

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(
        scrollregion=canvas.bbox("all")
    )
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

content_frame = scrollable_frame

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
image_refs = []


def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")
    modality = getattr(ds, "Modality", "Unknown")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()

    unique_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            modality = getattr(ds, "Modality", "Unknown")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), modality, getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in unique_cases:
                unique_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Modality": modality,
                    "Path": path,
                    "Ds": ds
                }

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{modality}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(unique_cases.values())
    update_table()


def update_table():
    for row in table_frame.winfo_children():
        row.destroy()

    headers = ["Name", "Date", "Modality", "CTDIvol (mGy)", "DLP (mGy·cm)", "Dose (mSv)", "Sex", "DOB", "Select"]
    for col, header in enumerate(headers):
        lbl = ctk.CTkLabel(table_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        lbl.grid(row=0, column=col, padx=6, pady=6)

    for i, data in enumerate(all_data, start=1):
        ctk.CTkLabel(table_frame, text=data['Name']).grid(row=i, column=0, padx=6)
        ctk.CTkLabel(table_frame, text=data['Date'].strftime("%Y-%m-%d")).grid(row=i, column=1, padx=6)
        ctk.CTkLabel(table_frame, text=data['Modality']).grid(row=i, column=2, padx=6)
        ctk.CTkLabel(table_frame, text=f"{data['CTDIvol']:.2f}").grid(row=i, column=3, padx=6)
        ctk.CTkLabel(table_frame, text=f"{data['DLP']:.2f}").grid(row=i, column=4, padx=6)
        ctk.CTkLabel(table_frame, text=f"{data['mSv']:.2f}").grid(row=i, column=5, padx=6)
        ctk.CTkLabel(table_frame, text=data['Sex']).grid(row=i, column=6, padx=6)
        ctk.CTkLabel(table_frame, text=data['DOB']).grid(row=i, column=7, padx=6)
        var = ctk.BooleanVar()
        chk = ctk.CTkCheckBox(table_frame, variable=var, command=lambda d=data, v=var: on_check(d, v))
        chk.grid(row=i, column=8)


def on_check(data, var):
    if var.get():
        if len(selected_cases) >= 4:
            messagebox.showwarning("Limit", "Select up to 4 cases only.")
            var.set(False)
        else:
            selected_cases.append(data)
    else:
        if data in selected_cases:
            selected_cases.remove(data)


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Warning", "Select exactly 2 or 4 cases to view.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases")
    top.geometry("1000x600")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        img_label = ctk.CTkLabel(frame, text="Image Not Available")
        try:
            ds = data["Ds"]
            if 'PixelData' in ds:
                arr = ds.pixel_array
                img = Image.fromarray(arr)
                img.thumbnail((200, 200))
                img_tk = ImageTk.PhotoImage(img)
                image_refs.append(img_tk)
                img_label.configure(image=img_tk, text="")
                img_label.image = img_tk
        except:
            pass
        img_label.pack(pady=5)

        info = f"""
👤 Name: {data['Name']}
🆔 ID: {data['StudyID']}
📅 Date: {data['Date'].strftime('%Y-%m-%d')}
🧬 Modality: {data['Modality']}
☢ Dose: {data['mSv']:.2f} mSv
🧪 CTDIvol: {data['CTDIvol']:.2f} mGy
📏 DLP: {data['DLP']:.2f} mGy·cm
⚧ Sex: {data['Sex']}
🎂 DOB: {data['DOB']}"""
        ctk.CTkLabel(frame, text=info, justify="left").pack()


root = ctk.CTk()
root.title("DICOM Elegant Viewer")
root.geometry("1300x850")

# Interactive Label
def animate():
    colors = ["#FF5733", "#33FF57", "#3357FF", "#F0A202"]
    current = animate.counter % len(colors)
    label.configure(text="Click here to load DICOM files", text_color=colors[current])
    animate.counter += 1
    root.after(500, animate)
animate.counter = 0

label = ctk.CTkLabel(root, text="Click here to load DICOM files", font=ctk.CTkFont(size=20, weight="bold"), cursor="hand2")
label.pack(pady=20)
label.bind("<Button-1>", lambda e: read_dicom_files())

# Table Frame
table_frame = ctk.CTkFrame(root)
table_frame.pack(fill="both", expand=True, padx=10, pady=10)

# Buttons
btn_frame = ctk.CTkFrame(root)
btn_frame.pack(pady=12)
ctk.CTkButton(btn_frame, text="View Selected Cases", command=show_selected_cases).pack(side="left", padx=10)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog, IntVar
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_vars = []  # to hold IntVars for checkboxes

# Convert DICOM to HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_vars.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table_data()
# زر جديد لعرض HL7 لجميع الحالات المختارة (2 أو 4)
def show_selected_hl7_messages():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    hl7_texts = []
    for data in selected:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            hl7_texts.append(f"--- HL7 for {data['Name']} ({data['Date'].strftime('%Y-%m-%d')}) ---\n{hl7}\n")
        else:
            hl7_texts.append(f"HL7 message not found for {data['Name']}.\n")

    hl7_window = ctk.CTkToplevel()
    hl7_window.title("HL7 Messages Viewer")
    hl7_window.geometry("800x600")
    hl7_window.attributes('-topmost', True)

    text_widget = ctk.CTkTextbox(hl7_window, width=780, height=580)
    text_widget.pack(padx=10, pady=10, fill="both", expand=True)
    text_widget.insert("0.0", "\n\n".join(hl7_texts))
    text_widget.configure(state="disabled")

# ... 

# فوق الجدول، نضيف زر HL7 مع الأزرار الأخرى
hl7_button = ctk.CTkButton(root, text="🗒 View HL7 for Selected", command=show_selected_hl7_messages)
hl7_button.place(relx=0.42, rely=0.03)
def display_table_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # headers
    headers = ["Select", "Name", "ID", "Date", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB", "HL7", "Delete"]

    # Sort data
    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # Create header labels
    for col, header in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="nsew")

    # Clear selected_vars and recreate
    selected_vars.clear()

    for row, data in enumerate(sorted_data, start=1):
        var = IntVar()
        selected_vars.append((var, data))
        checkbox = ctk.CTkCheckBox(content_frame, variable=var)
        checkbox.grid(row=row, column=0, padx=5, pady=5, sticky="nsew")

        ctk.CTkLabel(content_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["StudyID"]).grid(row=row, column=2, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=3, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=4, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']}").grid(row=row, column=5, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['DLP']}").grid(row=row, column=6, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row, column=7, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row, column=8, padx=5, pady=5, sticky="nsew")

        hl7_btn = ctk.CTkButton(content_frame, text="View HL7", width=80,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.grid(row=row, column=9, padx=5, pady=5)

        delete_btn = ctk.CTkButton(content_frame, text="Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.grid(row=row, column=10, padx=5, pady=5)

def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    # Also remove from selected_vars any var related to this data
    for var, d in selected_vars[:]:
        if d == data:
            selected_vars.remove((var, d))
    display_table_data()

def show_selected_cases():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.attributes('-topmost', True)  # Keep window on top

    for idx, data in enumerate(selected):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=14)).pack(pady=5, padx=10)

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1300x900")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_table_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

def load_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM Files", filetypes=[("DICOM files", "*.dcm")])
    if file_paths:
        # هنا تضيف كودك لقراءة ملفات DICOM وعرضها
        print("Loaded files:", file_paths)

root = ctk.CTk()
root.title("DICOM Viewer")
root.geometry("900x600")

# تحميل صورة الخلفية باستخدام CTkImage
bg_pil = Image.open("background.jpg").resize((900, 600))
bg_image = ctk.CTkImage(light_image=bg_pil, size=(900, 600))

bg_label = ctk.CTkLabel(root, image=bg_image)
bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)

# زر تحميل الملفات مع عرض وارتفاع في constructor
load_button = ctk.CTkButton(root, text="Load DICOM Files", width=150, height=35, command=load_dicom_files)
load_button.place(relx=0.02, rely=0.02)

# إطار للجدول مع لون شفاف قليلاً وتصميم أنيق
table_frame = ctk.CTkFrame(root, fg_color="#2c3e50", corner_radius=12)
table_frame.place(relx=0.05, rely=0.1, relwidth=0.9, relheight=0.85)

# هنا جدول بسيط مع رؤوس أعمدة وأمثلة بيانات
import tkinter as tk
columns = ["Select", "Patient Name", "Study Date", "Modality", "Dose"]

tree = ctk.CTkScrollableFrame(table_frame, corner_radius=10)
tree.pack(fill="both", expand=True, padx=10, pady=10)

# رأس الجدول
header_frame = ctk.CTkFrame(tree, fg_color=None)
header_frame.pack(fill="x", pady=(0,5))

for col in columns:
    lbl = ctk.CTkLabel(header_frame, text=col, width=120, anchor="w", font=ctk.CTkFont(size=14, weight="bold"))
    lbl.pack(side="left", padx=5)

# إضافة صف بيانات كمثال
def create_row(parent, data):
    row = ctk.CTkFrame(parent)
    row.pack(fill="x", pady=2)

    var = ctk.BooleanVar()
    cb = ctk.CTkCheckBox(row, variable=var)
    cb.pack(side="left", padx=5, ipadx=5)

    for i, val in enumerate(data):
        lbl = ctk.CTkLabel(row, text=val, width=120, anchor="w")
        lbl.pack(side="left", padx=5)
    return var

# بيانات تجريبية
sample_data = [
    ("John Doe", "2025-05-31", "CT", "5.2 mGy"),
    ("Maryam Ali", "2025-05-31", "X-Ray", "2.1 mGy"),
    ("Ahmed Samir", "2025-05-30", "MRI", "0 mGy"),
]

checkbox_vars = []
for row_data in sample_data:
    checkbox_vars.append(create_row(tree, row_data))

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("dark-blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
check_vars = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in temp_cases:
                img = None
                if 'PixelData' in ds:
                    img_pil = Image.fromarray(ds.pixel_array).convert("L")
                    img_pil.thumbnail((200, 200))
                    img = ctk.CTkImage(img_pil)

                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path,
                    "Modality": getattr(ds, "Modality", "Unknown"),
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{data_dict['StudyID']}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())
    display_text_data()
    animated_label.pack_forget()

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option] if sort_option != "Name" else x["Name"].lower())

    scroll_frame = ctk.CTkScrollableFrame(content_frame, corner_radius=10, fg_color="#ffffff")
    scroll_frame.pack(fill="both", expand=True, padx=20, pady=10)

    headers = ["Select", "Name", "Date", "Modality", "Study ID", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)"]
    for col, header in enumerate(headers):
        lbl = ctk.CTkLabel(scroll_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        lbl.grid(row=0, column=col, padx=10, pady=10, sticky="w")

    check_vars.clear()
    for row, data in enumerate(sorted_data, start=1):
        var = ctk.BooleanVar(value=data in selected_cases)
        check_vars.append((var, data))
        chk = ctk.CTkCheckBox(scroll_frame, variable=var)
        chk.grid(row=row, column=0, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(scroll_frame, text=data["Name"]).grid(row=row, column=1, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=2, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=data["Modality"]).grid(row=row, column=3, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["StudyID"])).grid(row=row, column=4, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=5, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["CTDIvol"])).grid(row=row, column=6, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["DLP"])).grid(row=row, column=7, padx=10, pady=5, sticky="w")

def update_selected_cases():
    selected_cases.clear()
    for var, data in check_vars:
        if var.get():
            selected_cases.append(data)

def show_hl7_message():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    data = selected_cases[-1]
    hl7_filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}.hl7"
    if os.path.exists(hl7_filename):
        with open(hl7_filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def show_selected_cases():
    update_selected_cases()
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, fg_color="#f0f0f0", corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"🧫 DLP: {data['DLP']} mGy·cm\n"
        )
        label = ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=14))
        label.pack(padx=10, pady=10)

    for i in range(2):
        top.grid_columnconfigure(i, weight=1)
    for i in range(2):
        top.grid_rowconfigure(i, weight=1)

def delete_selected_cases():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select cases to delete.")
        return
    for case in selected_cases:
        if case in all_data:
            all_data.remove(case)
    selected_cases.clear()
    display_text_data()

def on_sort_change():
    display_text_data()

def on_animated_label_click(event):
    read_dicom_files()

root = ctk.CTk()
root.title("DICOM Files Viewer with HL7 & CSV")
root.geometry("1200x800")

# خلفية بصورة
bg_image_path = "background.jpg"  # ضع هنا مسار صورتك للخلفية
if os.path.exists(bg_image_path):
    bg_pil = Image.open(bg_image_path)
    bg_pil = bg_pil.resize((1200, 800))
    bg_img = ImageTk.PhotoImage(bg_pil)
else:
    bg_img = None

background = ctk.CTkCanvas(root, width=1200, height=800)
background.pack(fill="both", expand=True)

if bg_img:
    background.create_image(0, 0, anchor="nw", image=bg_img)

# نستخدم Frame داخل الـ Canvas لسهولة وضع الودجت
main_frame = ctk.CTkFrame(root, fg_color="transparent")
main_frame.place(relx=0, rely=0, relwidth=1, relheight=1)

# Header frame
header_frame = ctk.CTkFrame(main_frame, height=60)
header_frame.pack(fill="x", pady=10, padx=10)

title_label = ctk.CTkLabel(header_frame, text="DICOM Files Viewer", font=ctk.CTkFont(size=20, weight="bold"))
title_label.pack(side="left", padx=10)

btn_load = ctk.CTkButton(header_frame, text="Load DICOM Files", command=read_dicom_files)
btn_load.pack(side="left", padx=10)

btn_delete = ctk.CTkButton(header_frame, text="Delete Selected", command=delete_selected_cases)
btn_delete.pack(side="left", padx=10)

sort_var = ctk.StringVar(value="Name")
sort_label = ctk.CTkLabel(header_frame, text="Sort by:")
sort_label.pack(side="left", padx=(30, 5))
sort_option_menu = ctk.CTkOptionMenu(header_frame, variable=sort_var, values=["Name", "Date"], command=lambda _: on_sort_change())
sort_option_menu.pack(side="left")

btn_show = ctk.CTkButton(header_frame, text="Show Selected Cases (2 or 4)", command=show_selected_cases)
btn_show.pack(side="right", padx=10)

btn_hl7 = ctk.CTkButton(header_frame, text="Show HL7 Message", command=show_hl7_message)
btn_hl7.pack(side="right", padx=10)

content_frame = ctk.CTkFrame(main_frame)
content_frame.pack(fill="both", expand=True, padx=20, pady=10)

animated_label = ctk.CTkLabel(content_frame, text="Click here to load DICOM files", font=ctk.CTkFont(size=24, weight="bold"), text_color="blue", cursor="hand2")
animated_label.pack(expand=True)
animated_label.bind("<Button-1>", on_animated_label_click)

def blink():
    current_color = animated_label.cget("text_color")
    new_color = "red" if current_color == "blue" else "blue"
    animated_label.configure(text_color=new_color)
    root.after(700, blink)

blink()

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("dark-blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
check_vars = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in temp_cases:
                img = None
                if 'PixelData' in ds:
                    img_pil = Image.fromarray(ds.pixel_array)
                    img_pil.thumbnail((300, 300))
                    img = ImageTk.PhotoImage(img_pil)

                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path,
                    "Modality": getattr(ds, "Modality", "Unknown"),
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{data_dict['StudyID']}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())
    display_text_data()

def display_text_data(filtered_data=None):
    for widget in content_frame.winfo_children():
        widget.destroy()

    data_to_show = filtered_data if filtered_data is not None else all_data

    sort_option = sort_var.get()
    sorted_data = sorted(data_to_show, key=lambda x: x[sort_option] if sort_option != "Name" else x["Name"].lower())

    scroll_frame = ctk.CTkScrollableFrame(content_frame, corner_radius=10, fg_color="#ffffff")
    scroll_frame.pack(fill="both", expand=True, padx=15, pady=10)

    headers = ["Select", "Name", "Date", "Modality", "Study ID", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)"]
    for col, header in enumerate(headers):
        lbl = ctk.CTkLabel(scroll_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        lbl.grid(row=0, column=col, padx=12, pady=8, sticky="w")

    check_vars.clear()

    # Gradient background colors for rows
    start_color = (230, 245, 255)  # light blue-ish
    end_color = (200, 220, 240)    # darker blue-ish
    n = len(sorted_data)
    def rgb_to_hex(rgb):
        return '#%02x%02x%02x' % rgb

    for row, data in enumerate(sorted_data, start=1):
        # Calculate gradient color for row background
        r = int(start_color[0] + (end_color[0] - start_color[0]) * row / max(n,1))
        g = int(start_color[1] + (end_color[1] - start_color[1]) * row / max(n,1))
        b = int(start_color[2] + (end_color[2] - start_color[2]) * row / max(n,1))
        bg_color = rgb_to_hex((r, g, b))

        # Create a frame for each row to have background color and borders
        row_frame = ctk.CTkFrame(scroll_frame, fg_color=bg_color, corner_radius=8)
        row_frame.grid(row=row, column=0, columnspan=len(headers), sticky="ew", padx=3, pady=2)
        row_frame.grid_columnconfigure(tuple(range(len(headers))), weight=1)

        var = ctk.BooleanVar(value=data in selected_cases)
        check_vars.append((var, data))
        chk = ctk.CTkCheckBox(row_frame, variable=var)
        chk.grid(row=0, column=0, padx=12, pady=5, sticky="w")

        ctk.CTkLabel(row_frame, text=data["Name"], anchor="w").grid(row=0, column=1, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=data["Date"].strftime("%Y-%m-%d"), anchor="w").grid(row=0, column=2, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=data["Modality"], anchor="w").grid(row=0, column=3, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=str(data["StudyID"]), anchor="w").grid(row=0, column=4, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=f"{data['mSv']:.2f}", anchor="w").grid(row=0, column=5, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=str(data["CTDIvol"]), anchor="w").grid(row=0, column=6, padx=12, pady=5, sticky="w")
        ctk.CTkLabel(row_frame, text=str(data["DLP"]), anchor="w").grid(row=0, column=7, padx=12, pady=5, sticky="w")

        # Add border lines by setting fg_color on the parent frame, row_frame handles bg

def update_selected_cases():
    selected_cases.clear()
    for var, data in check_vars:
        if var.get():
            selected_cases.append(data)

def show_hl7_message():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    data = selected_cases[-1]
    hl7_filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}.hl7"
    if os.path.exists(hl7_filename):
        with open(hl7_filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def show_selected_cases():
    update_selected_cases()
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, fg_color="#f0f0f0", corner_radius=12)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        # عرض الصورة بحجم أكبر
        if data["Image"]:
            # إعادة تحجيم الصورة للصورة أكبر في النافذة
            img_pil = Image.open(data["Path"])
            img_pil.thumbnail((400, 400))
            img_tk = ImageTk.PhotoImage(img_pil)
            img_label = ctk.CTkLabel(frame, image=img_tk, text="")
            img_label.image = img_tk
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=10)

    for i in range(2):
        top.grid_columnconfigure(i, weight=1)
        top.grid_rowconfigure(i, weight=1)

def search_by_id():
    search_id = search_entry.get().strip()
    if not search_id:
        display_text_data()
        return

    filtered = [d for d in all_data if str(d["StudyID"]) == search_id]
    if not filtered:
        messagebox.showinfo("Search Result", f"No records found for ID: {search_id}")
    display_text_data(filtered)

def resize_bg(event):
    global bg_img_resized, bg_label
    new_width = event.width
    new_height = event.height
    resized = bg_img_orig.resize((new_width, new_height), Image.ANTIALIAS)
    bg_img_resized = ImageTk.PhotoImage(resized)
    bg_label.configure(image=bg_img_resized)

root = ctk.CTk()
root.title("DICOM Viewer with Search and Gradient Table")
root.geometry("1000x650")
root.minsize(900, 600)

# خلفية الصورة
bg_img_orig = Image.open("g.jpg")
bg_img_resized = ImageTk.PhotoImage(bg_img_orig.resize((1000, 650)))
bg_label = ctk.CTkLabel(root, image=bg_img_resized)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
root.bind("<Configure>", resize_bg)

# متغيرات الترتيب والبحث
sort_var = ctk.StringVar(value="Name")

# بحث بالـ ID
search_frame = ctk.CTkFrame(root, fg_color="transparent")
search_frame.place(relx=0.02, rely=0.02, relwidth=0.25, height=40)

search_entry = ctk.CTkEntry(search_frame, placeholder_text="Search by Study ID...")
search_entry.pack(side="left", fill="both", expand=True, padx=(10, 0), pady=5)

search_btn = ctk.CTkButton(search_frame, text="🔍", width=40, command=search_by_id)
search_btn.pack(side="left", padx=10, pady=5)

# زر فرز
sort_option_menu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Name", "Date"])
sort_option_menu.place(relx=0.3, rely=0.02)

# زر عرض الحالات المختارة
show_button = ctk.CTkButton(root, text="Show Selected Cases", command=show_selected_cases, width=160)
show_button.place(relx=0.48, rely=0.02)

# زر حذف الحالات المختارة
def delete_selected():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select cases to delete.")
        return
    for sel in selected_cases:
        if sel in all_data:
            all_data.remove(sel)
    selected_cases.clear()
    display_text_data()

delete_button = ctk.CTkButton(root, text="Delete Selected", command=delete_selected, width=140)
delete_button.place(relx=0.65, rely=0.02)

# زر عرض HL7
hl7_button = ctk.CTkButton(root, text="Show HL7 Message", command=show_hl7_message, width=140)
hl7_button.place(relx=0.80, rely=0.02)

# محتوى البيانات مع جدول
content_frame = ctk.CTkFrame(root, fg_color="#ffffff", corner_radius=15)
content_frame.place(relx=0.02, rely=0.08, relwidth=0.96, relheight=0.9)

welcome_label = ctk.CTkLabel(content_frame, text="Click here to select DICOM files",
                            text_color="blue", font=ctk.CTkFont(size=20, weight="bold"), cursor="hand2")
welcome_label.pack(expand=True)

def on_welcome_click(event):
    read_dicom_files()

welcome_label.bind("<Button-1>", on_welcome_click)

root.mainloop()
